#Getting Enviornment Set

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist//spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar -xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'/content/spark-2.4.5-bin-hadoop2.7'

<h1> Major imports </h1>

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType, IntegerType, StringType, FloatType
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
from datetime import date
import calendar
import math

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


<h1> Starting Spark Session </h1>

In [ ]:
conf = pyspark.SparkConf().setAppName('CaseStudy1').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

<h2> Loading Data </h2>

In [ ]:
train_df = spark.read.format("csv").option("header", "true").load('/content/drive/My Drive/Grocery/train.csv')
test_df = spark.read.format("csv").option("header", "true").load('/content/drive/My Drive/Grocery/test.csv')
transactions_df = spark.read.format("csv").option("header", "true").load('/content/drive/My Drive/Grocery/transactions.csv')
stores_df = spark.read.format("csv").option("header", "true").load('/content/drive/My Drive/Grocery/stores.csv')
oil_df = spark.read.format("csv").option("header", "true").load('/content/drive/My Drive/Grocery/oil.csv')
items_df = spark.read.format("csv").option("header", "true").load('/content/drive/My Drive/Grocery/items.csv')
holidays_events_df = spark.read.format("csv").option("header", "true").load('/content/drive/My Drive/Grocery/holidays_events.csv')

Creating Views for writing SQL queries....

In [ ]:
train_df = train_df.withColumn('id', col('id').cast(IntegerType())).withColumn('date', col('date').cast(DateType())).withColumn('store_nbr', col('store_nbr').cast(IntegerType())).withColumn('item_nbr', col('item_nbr').cast(IntegerType())).withColumn('unit_sales', col('unit_sales').cast(FloatType())).withColumn('onpromotion', col('onpromotion').cast(StringType()))
stores_df = stores_df.withColumn('store_nbr', col('store_nbr').cast(IntegerType())).withColumn('cluster', col('cluster').cast(IntegerType()))
items_df = items_df.withColumn('item_nbr', col('item_nbr').cast(IntegerType())).withColumn('class', col('class').cast(IntegerType())).withColumn('perishable', col('perishable').cast(IntegerType()))
transactions_df = transactions_df.withColumn('date', col('date').cast(DateType())).withColumn('store_nbr', col('store_nbr').cast(IntegerType())).withColumn('transactions', col('transactions').cast(IntegerType()))
holidays_events_df= holidays_events_df.withColumn('date', col('date').cast(DateType()))
test_df = test_df.withColumn('date', col('date').cast(DateType())).withColumn('store_nbr', col('store_nbr').cast(IntegerType())).withColumn('item_nbr', col('item_nbr').cast(IntegerType())).withColumn('onpromotion', col('onpromotion').cast(StringType()))

In [ ]:
train_df.createOrReplaceTempView('data')
train_df.cache

stores_df.createOrReplaceTempView('store_data')
stores_df.cache

items_df.createOrReplaceTempView('items_data')
items_df.cache

transactions_df.createOrReplaceTempView('transactions_data')
transactions_df.cache

holidays_events_df.createOrReplaceTempView('holidays_data')
holidays_events_df.cache

test_df.createOrReplaceTempView('test_view')
test_df.cache

<bound method DataFrame.cache of DataFrame[id: string, date: date, store_nbr: int, item_nbr: int, onpromotion: string]>

#Data Collection

Before building features and stacking data from other views, 2 things we are considering:

1. From the EDA we did saw that oil prices didn't have any impact on the sales, so oil prices will be skipped from our final data that we will build for modelling purpose
2. The onpromotion field was for the starting days, we can say that at that point they might not be tracking promotions, we have 2 option either, we can give a value of 2(which will give us info that promotions were not tracked), since we have a lot of data, and we are considering data at day level, considering data with data points only for 2017.
3. Eliminating negative values, making them zero and chaning onpromotion field to 0 and 1, where 0 is not on promotion and 1 is on promotion
4. Normaizing unit sales with log(x + 1), as for real-valued input, log(x + 1) is accurate, also for x so small that 1 + x == 1 in floating-point accuracy.

In [ ]:
train_df = spark.sql('''SELECT date, store_nbr, item_nbr, CASE WHEN unit_sales < 0 THEN 0 ELSE LOG(unit_sales + 1) END as unit_sales, 
                        CASE WHEN onpromotion = \'True\' THEN 1 ELSE 0 END as onpromotion FROM data WHERE date >= \'2017-01-01\'''')

In [ ]:
train_df.cache

<bound method DataFrame.cache of DataFrame[date: date, store_nbr: int, item_nbr: int, unit_sales: double, onpromotion: int]>

In [ ]:
train_df.show()

+----------+---------+--------+------------------+-----------+
|      date|store_nbr|item_nbr|        unit_sales|onpromotion|
+----------+---------+--------+------------------+-----------+
|2017-01-01|       25|   99197|0.6931471805599453|          0|
|2017-01-01|       25|  103665|2.0794415416798357|          0|
|2017-01-01|       25|  105574|0.6931471805599453|          0|
|2017-01-01|       25|  105857|1.6094379124341003|          0|
|2017-01-01|       25|  106716|1.0986122886681098|          0|
|2017-01-01|       25|  108698|1.0986122886681098|          0|
|2017-01-01|       25|  108786|0.6931471805599453|          0|
|2017-01-01|       25|  108797|0.6931471805599453|          0|
|2017-01-01|       25|  108862|0.6931471805599453|          0|
|2017-01-01|       25|  108952|1.0986122886681098|          0|
|2017-01-01|       25|  114790|1.0986122886681098|          0|
|2017-01-01|       25|  114800|1.9459101490553132|          0|
|2017-01-01|       25|  115267|1.0986122886681098|     

In [ ]:
#final_df.coalesce(1).write.option("header","true").option("sep",",").mode("overwrite").csv("train_df")

In [ ]:
print('Total number of rows in train: {}'.format(train_df.count()))

Total number of rows in train: 23808261


In [ ]:
#final_df.createOrReplaceTempView('final_data')
#final_df.cache

<bound method DataFrame.cache of DataFrame[date: date, store_nbr: int, item_nbr: int, unit_sales: double, onpromotion: int]>

**Since we are conisdering data daily, and we want to convert our time series problem to a supervised learning problem, we will be using the Windowing method to do so, what we do here is we unstack time from rows to column with each column representing a timestep**

One important update that we need to consider here is that, we don't have information about stock outs, so for store-item level, we will be seeing null values when windowing data, we are going fill all null with 0 for me, we may smooth it later, but this can give us some information on the stock out/not on shelf, but again this is an assumption and we are going forward with this for now.

<h2> Data collection using train file a item-store level </h2>

<h3> Using Sales Data </h3>

In [ ]:
train_sales_df = train_df.groupby('store_nbr', 'item_nbr').pivot('date').sum('unit_sales').sort('store_nbr', 'item_nbr').fillna(0)

In [ ]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")
spark.conf.set("spark.sql.crossJoin.enabled", "true")

In [ ]:
print(train_sales_df.toPandas().shape)

(167515, 229)


This 167515 is the total number of item-store combination we have in our sales data, we may train our models with the same number of data points if we continue with this approach of stacking information from all our files....

In [ ]:
train_sales_df.cache

<bound method DataFrame.cache of DataFrame[store_nbr: int, item_nbr: int, 2017-01-01: double, 2017-01-02: double, 2017-01-03: double, 2017-01-04: double, 2017-01-05: double, 2017-01-06: double, 2017-01-07: double, 2017-01-08: double, 2017-01-09: double, 2017-01-10: double, 2017-01-11: double, 2017-01-12: double, 2017-01-13: double, 2017-01-14: double, 2017-01-15: double, 2017-01-16: double, 2017-01-17: double, 2017-01-18: double, 2017-01-19: double, 2017-01-20: double, 2017-01-21: double, 2017-01-22: double, 2017-01-23: double, 2017-01-24: double, 2017-01-25: double, 2017-01-26: double, 2017-01-27: double, 2017-01-28: double, 2017-01-29: double, 2017-01-30: double, 2017-01-31: double, 2017-02-01: double, 2017-02-02: double, 2017-02-03: double, 2017-02-04: double, 2017-02-05: double, 2017-02-06: double, 2017-02-07: double, 2017-02-08: double, 2017-02-09: double, 2017-02-10: double, 2017-02-11: double, 2017-02-12: double, 2017-02-13: double, 2017-02-14: double, 2017-02-15: double, 2017-0

<h3> Using promo data </h3>

Now test files does have promotion details, so adding that information to create the promo df.

In [ ]:
train_promo_df = train_df.groupby('store_nbr', 'item_nbr').pivot('date').sum('onpromotion').sort('store_nbr', 'item_nbr').fillna(0)

In [ ]:
print(train_promo_df.toPandas().shape)

(167515, 229)


In [ ]:
train_promo_df.cache

<bound method DataFrame.cache of DataFrame[store_nbr: int, item_nbr: int, 2017-01-01: bigint, 2017-01-02: bigint, 2017-01-03: bigint, 2017-01-04: bigint, 2017-01-05: bigint, 2017-01-06: bigint, 2017-01-07: bigint, 2017-01-08: bigint, 2017-01-09: bigint, 2017-01-10: bigint, 2017-01-11: bigint, 2017-01-12: bigint, 2017-01-13: bigint, 2017-01-14: bigint, 2017-01-15: bigint, 2017-01-16: bigint, 2017-01-17: bigint, 2017-01-18: bigint, 2017-01-19: bigint, 2017-01-20: bigint, 2017-01-21: bigint, 2017-01-22: bigint, 2017-01-23: bigint, 2017-01-24: bigint, 2017-01-25: bigint, 2017-01-26: bigint, 2017-01-27: bigint, 2017-01-28: bigint, 2017-01-29: bigint, 2017-01-30: bigint, 2017-01-31: bigint, 2017-02-01: bigint, 2017-02-02: bigint, 2017-02-03: bigint, 2017-02-04: bigint, 2017-02-05: bigint, 2017-02-06: bigint, 2017-02-07: bigint, 2017-02-08: bigint, 2017-02-09: bigint, 2017-02-10: bigint, 2017-02-11: bigint, 2017-02-12: bigint, 2017-02-13: bigint, 2017-02-14: bigint, 2017-02-15: bigint, 2017-0

In [ ]:
test_df = spark.sql('SELECT date, store_nbr, item_nbr, CASE WHEN onpromotion = \'True\' THEN 1 ELSE 0 END AS onpromotion FROM test_view')

In [ ]:
test_promo_df = test_df.groupby('store_nbr', 'item_nbr').pivot('date').sum('onpromotion').sort('store_nbr', 'item_nbr').fillna(0)

In [ ]:
print(test_promo_df.toPandas().shape)

(210654, 18)


test_promo has more number of rows than that of train, but we will stack only the rows that are in train...

In [ ]:
test_promo_df.cache

<bound method DataFrame.cache of DataFrame[store_nbr: int, item_nbr: int, 2017-08-16: bigint, 2017-08-17: bigint, 2017-08-18: bigint, 2017-08-19: bigint, 2017-08-20: bigint, 2017-08-21: bigint, 2017-08-22: bigint, 2017-08-23: bigint, 2017-08-24: bigint, 2017-08-25: bigint, 2017-08-26: bigint, 2017-08-27: bigint, 2017-08-28: bigint, 2017-08-29: bigint, 2017-08-30: bigint, 2017-08-31: bigint]>

In [ ]:
#creating the list of items part of test_promo
train_promo_store_item_lst = [(i[0], i[1]) for i in train_promo_df.select('store_nbr', 'item_nbr').collect()]

In [ ]:
len(train_promo_store_item_lst)

167515

In [ ]:
train_promo_store_item_lst_str = [",".join([str(x) for x in item]) for item in train_promo_store_item_lst]

In [ ]:
import pyspark.sql.functions as f

In [ ]:
#filtering rows from the test_promo
test_promo_df = test_promo_df.withColumn("combined_id", f.concat(f.col("store_nbr"), f.lit(","), f.col("item_nbr")))\
    .where(f.col("combined_id").isin(train_promo_store_item_lst_str))

In [ ]:
test_promo_df = test_promo_df.drop('combined_id')

In [ ]:
train_promo_df = train_promo_df.toPandas()
test_promo_df = test_promo_df.toPandas()

In [ ]:
final_promo_df = train_promo_df.merge(test_promo_df, on = ['store_nbr', 'item_nbr'], how = 'left').fillna(0)

In [ ]:
final_promo_df = train_promo_df.join(test_promo_df, on=['store_nbr', 'item_nbr'], how='left').fillna(0)

In [ ]:
train_sales_df = train_sales_df.toPandas()

In [ ]:
print('Shape of sales df is {}'.format(train_sales_df.shape))
print('Shape of promo df is {}'.format(final_promo_df.shape))

Shape of sales df is (167515, 229)
Shape of promo df is (167515, 245)


So we have created df such that each row correspond to sales and promotion details for item-store level.
We will gather this detail at store level and item level separately in next few cells...

In [ ]:
train_sales_df.head()

,store_nbr,item_nbr,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-17,2017-01-18,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,2017-01-25,2017-01-26,2017-01-27,2017-01-28,2017-01-29,2017-01-30,2017-01-31,2017-02-01,2017-02-02,2017-02-03,2017-02-04,2017-02-05,2017-02-06,2017-02-07,...,2017-07-07,2017-07-08,2017-07-09,2017-07-10,2017-07-11,2017-07-12,2017-07-13,2017-07-14,2017-07-15,2017-07-16,2017-07-17,2017-07-18,2017-07-19,2017-07-20,2017-07-21,2017-07-22,2017-07-23,2017-07-24,2017-07-25,2017-07-26,2017-07-27,2017-07-28,2017-07-29,2017-07-30,2017-07-31,2017-08-01,2017-08-02,2017-08-03,2017-08-04,2017-08-05,2017-08-06,2017-08-07,2017-08-08,2017-08-09,2017-08-10,2017-08-11,2017-08-12,2017-08-13,2017-08-14,2017-08-15
0,1,96995,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.386294,1.098612,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.098612,0.000000,0.693147,1.098612,0.000000,0.000000,1.098612,1.098612,0.000000,0.000000,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,99197,0.0,0.000000,1.386294,0.693147,0.693147,0.693147,1.098612,0.000000,0.000000,0.693147,0.000000,0.000000,0.000000,1.098612,0.693147,0.000000,0.000000,0.000000,1.098612,0.000000,1.386294,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147,0.000000,0.000000,0.000000,1.098612,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.693147,0.000000,0.693147,0.000000,0.693147,1.098612,0.000000,1.386294,1.098612,0.000000,0.693147,1.386294,1.098612,0.693147,1.791759,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.000000,1.098612,0.000000,1.098612,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1,103520,0.0,0.693147,1.098612,0.000000,1.098612,1.386294,0.693147,0.000000,0.693147,0.693147,0.000000,0.000000,0.693147,0.000000,1.098612,1.098612,0.000000,1.098612,0.693147,1.386294,1.609438,0.000000,0.000000,0.693147,1.098612,0.693147,1.098612,0.000000,0.000000,0.000000,0.693147,1.098612,0.693147,0.000000,1.098612,1.098612,0.693147,0.000000,...,1.386294,0.693147,0.000000,0.000000,1.098612,0.693147,1.386294,1.945910,0.693147,0.000000,1.098612,0.000000,0.693147,0.693147,0.000000,1.386294,0.693147,1.386294,1.098612,0.693147,1.098612,1.098612,1.386294,0.000000,0.693147,1.386294,0.693147,1.098612,1.386294,0.000000,0.000000,0.000000,1.386294,0.000000,1.386294,0.693147,0.693147,0.693147,0.000000,0.000000
3,1,103665,0.0,0.000000,0.000000,1.386294,1.098612,1.098612,0.693147,1.098612,0.000000,2.079442,0.693147,0.693147,2.079442,0.000000,0.000000,1.098612,0.693147,1.791759,1.386294,1.098612,1.609438,1.098612,1.386294,1.098612,1.098612,1.098612,1.945910,1.945910,0.000000,0.000000,0.693147,0.693147,1.945910,1.386294,1.791759,0.000000,0.693147,1.945910,...,1.098612,2.302585,0.000000,1.386294,1.098612,1.098612,0.693147,2.197225,0.000000,0.000000,0.000000,1.791759,1.386294,0.000000,0.000000,1.098612,0.693147,1.609438,1.386294,1.609438,0.000000,1.098612,1.609438,1.098612,1.098612,1.609438,1.098612,1.098612,2.197225,1.386294,0.693147,1.098612,0.000000,2.079442,2.302585,1.098612,0.000000,0.000000,0.693147,0.693147
4,1,105574,0.0,0.000000,1.791759,2.564949,2.302585,1.945910,1.609438,1.098612,1.386294,2.302585,2.484907,1.386294,1.098612,1.386294,0.000000,1.609438,2.397895,2.197225,2.639057,1.609438,1.609438,1.098612,2.079442,1.945910,1.945910,2.079442,2.484907,1.609438,0.693147,2.197

In [ ]:
final_promo_df.head()

,store_nbr,item_nbr,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-17,2017-01-18,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,2017-01-25,2017-01-26,2017-01-27,2017-01-28,2017-01-29,2017-01-30,2017-01-31,2017-02-01,2017-02-02,2017-02-03,2017-02-04,2017-02-05,2017-02-06,2017-02-07,...,2017-07-23,2017-07-24,2017-07-25,2017-07-26,2017-07-27,2017-07-28,2017-07-29,2017-07-30,2017-07-31,2017-08-01,2017-08-02,2017-08-03,2017-08-04,2017-08-05,2017-08-06,2017-08-07,2017-08-08,2017-08-09,2017-08-10,2017-08-11,2017-08-12,2017-08-13,2017-08-14,2017-08-15,2017-08-16,2017-08-17,2017-08-18,2017-08-19,2017-08-20,2017-08-21,2017-08-22,2017-08-23,2017-08-24,2017-08-25,2017-08-26,2017-08-27,2017-08-28,2017-08-29,2017-08-30,2017-08-31
0,1,96995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,99197,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,103520,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,103665,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,105574,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<h2> Data Collection using item file </h2>

In [ ]:
items_df = items_df.toPandas()

In [ ]:
items_df.head()

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


Adding item info to our sales df to form data points at item level....

In [ ]:
items_sales_df = train_sales_df.drop('store_nbr', axis =1).groupby('item_nbr').sum().reset_index()

In [ ]:
items_promo_df = final_promo_df.drop('store_nbr', axis =1).groupby('item_nbr').sum().reset_index()

In [ ]:
items_promo_df.head()

,item_nbr,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-17,2017-01-18,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,2017-01-25,2017-01-26,2017-01-27,2017-01-28,2017-01-29,2017-01-30,2017-01-31,2017-02-01,2017-02-02,2017-02-03,2017-02-04,2017-02-05,2017-02-06,2017-02-07,2017-02-08,...,2017-07-23,2017-07-24,2017-07-25,2017-07-26,2017-07-27,2017-07-28,2017-07-29,2017-07-30,2017-07-31,2017-08-01,2017-08-02,2017-08-03,2017-08-04,2017-08-05,2017-08-06,2017-08-07,2017-08-08,2017-08-09,2017-08-10,2017-08-11,2017-08-12,2017-08-13,2017-08-14,2017-08-15,2017-08-16,2017-08-17,2017-08-18,2017-08-19,2017-08-20,2017-08-21,2017-08-22,2017-08-23,2017-08-24,2017-08-25,2017-08-26,2017-08-27,2017-08-28,2017-08-29,2017-08-30,2017-08-31
0,96995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,99197,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,103501,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0
3,103520,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,103665,0,0,0,0,0,0,0,13,0,0,0,0,0,0,14,0,0,0,0,0,0,11,0,0,0,0,0,0,13,0,0,0,0,0,0,12,0,0,0,...,11,0,0,0,0,0,0,11,0,0,0,0,0,0,10,0,0,0,0,0,0,10,0,0,0.0,0.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,0.0,1.0,0.0


In [ ]:
items_sales_df.head()

,item_nbr,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-17,2017-01-18,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,2017-01-25,2017-01-26,2017-01-27,2017-01-28,2017-01-29,2017-01-30,2017-01-31,2017-02-01,2017-02-02,2017-02-03,2017-02-04,2017-02-05,2017-02-06,2017-02-07,2017-02-08,...,2017-07-07,2017-07-08,2017-07-09,2017-07-10,2017-07-11,2017-07-12,2017-07-13,2017-07-14,2017-07-15,2017-07-16,2017-07-17,2017-07-18,2017-07-19,2017-07-20,2017-07-21,2017-07-22,2017-07-23,2017-07-24,2017-07-25,2017-07-26,2017-07-27,2017-07-28,2017-07-29,2017-07-30,2017-07-31,2017-08-01,2017-08-02,2017-08-03,2017-08-04,2017-08-05,2017-08-06,2017-08-07,2017-08-08,2017-08-09,2017-08-10,2017-08-11,2017-08-12,2017-08-13,2017-08-14,2017-08-15
0,96995,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,8.658693,8.723231,9.128696,4.564348,5.545177,6.068426,5.545177,6.931472,9.351840,7.336937,3.583519,6.761573,8.030084,6.356108,4.852030,7.336937,6.761573,6.643790,6.761573,6.356108,6.356108,7.336937,9.469623,5.950643,6.173786,5.950643,9.821844,9.534161,6.356108,7.336937,5.662960,7.454720,2.484907,5.950643,5.545177,8.841014,7.742402,4.969813,7.167038,7.742402
1,99197,0.693147,17.422746,16.604036,20.569303,16.203025,16.278613,14.775909,17.317386,14.986630,15.833927,15.951710,6.761573,14.293482,11.926359,15.825731,16.818830,14.881269,11.079061,14.151754,17.289215,15.931091,11.010068,6.356108,7.649693,13.523146,5.375278,4.969813,5.257495,3.871201,1.386294,4.276666,1.791759,2.197225,2.079442,0.000000,1.386294,0.693147,0.693147,1.386294,...,15.951710,12.955978,16.822064,12.712215,12.984149,12.529894,9.798127,16.834959,17.820431,12.635254,21.573849,10.227309,18.940069,12.999897,18.164683,9.416378,15.890269,14.593587,11.325921,12.214041,10.044987,13.746289,8.147867,9.534161,7.677864,5.257495,3.583519,2.079442,1.386294,3.178054,3.178054,4.969813,3.178054,4.969813,2.079442,2.197225,1.386294,2.079442,0.000000,0.000000
2,103501,0.000000,55.868320,54.627085,42.810313,39.555298,35.717635,47.208504,47.542538,40.189274,39.200893,42.293954,30.419318,38.025918,47.754808,47.340190,34.580801,39.024766,36.030619,30.807906,31.939143,47.582552,45.467144,38.906258,40.548321,35.780320,33.771725,32.762725,49.361495,45.503756,41.835181,43.802522,46.101695,43.106860,37.214161,51.808490,48.945795,42.174649,38.635681,33.966697,...,30.224574,41.855801,44.100480,40.829176,38.149053,30.873573,29.474856,30.630039,42.327658,39.961190,38.351096,39.969227,36.780108,31.797644,29.056968,36.082262,47.370206,43.221471,34.026782,28.355624,29.426066,36.074726,43.624993,48.269842,42.539276,47.547413,41.086834,36.374643,33.346057,40.030669,38.578235,33.531460,35.296421,35.584104,26.270815,32.776619,34.416498,36.546914,34.773173,35.512841
3,103520,0.000000,38.875486,35.822995,34.979211,42.252967,51.397412,49.505990,33.846832,33.336007,31.741073,33.816061,35.948747,38.145971,44.153184,30.050220,31.574500,34.683561,32.486337,32.316438,36.957747,47.076410,32.380976,24.143497,27.650055,28.469182,36.252177,43.549649,41.534746,33.653942,25.955876,31.456717,40.882192,31.452263,39.750767,38.587804,31.846434,25.801725,28.132481,32.527159,...,44.792362,46.504559,43.950660,38.940025,36.690496,39.126800,31.448937,42.495373,50.935376,36.855941,33.420748,30.735399,44.266741,37.761354,50.100501,49.240780,41.534746,39.178916,42.442840,40.489378,45.190111,52.024167,49.228358,32.203109,35.291967,51.361044,33.589403,43.176097,47.286190,58.097702,35.630624,32.567752,47.213872,41.198030,43.569852,48.697330,47.015385,39.070042,33.798042,40.030669
4,103665,2.079442,56.22540

In [ ]:
items_sales_df.shape, items_promo_df.shape

((4018, 228), (4018, 244))

<h2> Data collection using store file </h2>

In [ ]:
stores_df = stores_df.toPandas()

In [ ]:
stores_df.columns

Index(['store_nbr', 'state', 'type', 'cluster'], dtype='object')

One thing we saw from EDA was that city & state add same sort of information in terms of total sales and we can use only one of these for our model

In [ ]:
stores_df = stores_df.drop('city', axis = 1)

In [ ]:
stores_df.head()

,store_nbr,state,type,cluster
0,1,Pichincha,D,13
1,2,Pichincha,D,13
2,3,Pichincha,D,8
3,4,Pichincha,D,9
4,5,Santo Domingo de los Tsachilas,D,4


In [ ]:
store_sales_df = train_sales_df.drop('item_nbr', axis =1).groupby('store_nbr').sum().reset_index()

In [ ]:
store_promo_df = final_promo_df.drop('item_nbr', axis =1).groupby('store_nbr').sum().reset_index()

In [ ]:
store_sales_df.shape, store_promo_df.shape

((54, 228), (54, 244))

In [ ]:
store_sales_df.head()

,store_nbr,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-17,2017-01-18,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,2017-01-25,2017-01-26,2017-01-27,2017-01-28,2017-01-29,2017-01-30,2017-01-31,2017-02-01,2017-02-02,2017-02-03,2017-02-04,2017-02-05,2017-02-06,2017-02-07,2017-02-08,...,2017-07-07,2017-07-08,2017-07-09,2017-07-10,2017-07-11,2017-07-12,2017-07-13,2017-07-14,2017-07-15,2017-07-16,2017-07-17,2017-07-18,2017-07-19,2017-07-20,2017-07-21,2017-07-22,2017-07-23,2017-07-24,2017-07-25,2017-07-26,2017-07-27,2017-07-28,2017-07-29,2017-07-30,2017-07-31,2017-08-01,2017-08-02,2017-08-03,2017-08-04,2017-08-05,2017-08-06,2017-08-07,2017-08-08,2017-08-09,2017-08-10,2017-08-11,2017-08-12,2017-08-13,2017-08-14,2017-08-15
0,1,0.0,1909.598470,3734.738494,3721.581691,3381.289871,3405.684219,3267.285339,2105.142204,3305.426145,3250.739364,3447.712170,2979.834257,3336.075177,3305.091910,2032.795322,3215.315888,3216.161028,3439.817063,3003.688901,2904.241102,3462.520626,1876.459417,3311.552974,3156.877705,3370.657287,2989.120303,3279.334903,3249.767222,1870.487130,3057.897599,2986.693481,3635.996898,3163.007265,3322.700522,3435.280176,1864.332386,3066.860419,3226.849200,3357.844082,...,3229.158248,3091.110890,1825.650667,3149.056253,2998.543876,3342.143178,2861.442996,3227.451876,2877.485060,1658.464211,3098.552775,3055.288388,3252.108179,3021.702843,3258.461440,3071.833927,1843.189280,3048.267330,2932.001833,3177.864957,2869.360471,3089.007150,2927.337820,1760.387826,3149.267643,3172.488334,3471.598178,2695.363317,3166.067255,2795.193639,1694.877666,3006.549491,2994.871649,3134.129057,3049.077967,1804.257980,2388.263524,1526.721366,2986.071170,2865.330319
1,2,0.0,4917.580925,4215.896120,4179.835998,3633.582513,4091.651346,4416.109988,4676.830677,3771.890228,3366.022875,3702.321130,3220.523266,3887.560403,4408.016818,4545.188050,3727.271406,3616.588166,3721.078838,3313.588577,3570.860160,4573.944284,4867.876854,3771.670114,3580.224686,3860.032837,3196.638083,3720.003995,4398.079379,4547.412096,3471.556806,3875.333731,4235.811828,3707.590522,3827.477277,4396.645832,4487.991163,3766.864717,3645.521151,3842.001682,...,3649.447463,4089.896782,4194.713004,3525.533569,3249.760821,3362.226068,3109.507248,3501.219805,4062.535314,4193.766918,3478.021829,3297.115729,3559.492553,3241.074745,3685.962424,3946.021388,4018.783863,3545.245766,3277.977174,3510.463780,3153.448233,3672.773615,3953.856607,4066.401435,3558.481984,3663.508045,3897.669890,3349.317115,3644.980801,3714.208923,4080.147177,3381.292277,3210.191738,3356.708311,2967.608277,3399.444116,3397.514971,3598.581210,3321.329992,3152.816676
2,3,0.0,7122.511109,6226.395166,6144.813040,5641.845924,6099.497740,6790.194070,6683.576204,5571.781010,5266.053988,5510.941909,5022.961126,5734.942652,6415.481254,6802.531678,5679.060945,5122.896951,5545.210152,4986.949141,5445.903628,6725.173607,6877.878234,5738.201772,5194.375852,5497.256811,4893.947857,5734.163938,6363.126727,6390.421078,5141.913974,5380.663812,6130.468595,5219.224109,6057.323459,6582.023931,6718.419130,5590.517077,5295.259647,5720.473853,...,5722.010229,6147.289477,6318.654974,5392.751244,5246.000120,5379.544505,4697.767805,5434.674017,6110.822307,6245.597287,5469.358272,5112.219057,5340.076528,5057.205033,5530.142142,5965.029468,6127.453272,5411.115018,4993.040933,5240.677686,4876.490287,5517.761064,5918.215265,6221.139873,5382.219782,5588.402978,5773.066411,5116.130107,5686.533099,5908.555123,6067.029411,5301.267398,5031.745401,5181.878200,4784.015093,5415.157390,5330.343216,5613.627189,5286.741718,5134.690221
3,4,0.0,4729.934125,4120.414355,3914.114557,3479.424707,3761.194571,4200.497033,4560.455803,3468.428773,3167.510541,3466.928979,2767.442308,3519.928551,4037.717587,4399.282788,3571.486107,3007.661558,3615.324798,2856.414160,3228.853576,4232.718982,4605.966189,3613.508863,

In [ ]:
store_promo_df.head()

,store_nbr,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-17,2017-01-18,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,2017-01-25,2017-01-26,2017-01-27,2017-01-28,2017-01-29,2017-01-30,2017-01-31,2017-02-01,2017-02-02,2017-02-03,2017-02-04,2017-02-05,2017-02-06,2017-02-07,2017-02-08,...,2017-07-23,2017-07-24,2017-07-25,2017-07-26,2017-07-27,2017-07-28,2017-07-29,2017-07-30,2017-07-31,2017-08-01,2017-08-02,2017-08-03,2017-08-04,2017-08-05,2017-08-06,2017-08-07,2017-08-08,2017-08-09,2017-08-10,2017-08-11,2017-08-12,2017-08-13,2017-08-14,2017-08-15,2017-08-16,2017-08-17,2017-08-18,2017-08-19,2017-08-20,2017-08-21,2017-08-22,2017-08-23,2017-08-24,2017-08-25,2017-08-26,2017-08-27,2017-08-28,2017-08-29,2017-08-30,2017-08-31
0,1,0,128,197,479,152,358,156,134,166,168,381,159,371,185,150,169,188,418,158,323,175,119,167,173,425,111,303,121,97,131,123,385,104,296,133,119,120,137,418,...,120,149,156,369,145,291,133,107,143,129,364,95,290,110,83,119,113,338,125,216,103,73,120,115,446.0,113.0,276.0,121.0,88.0,118.0,103.0,363.0,130.0,294.0,132.0,88.0,129.0,133.0,369.0,153.0
1,2,0,237,199,508,168,348,192,194,163,176,394,171,363,198,184,193,181,436,170,350,187,199,191,182,436,144,308,155,136,127,152,394,130,305,149,150,162,141,459,...,199,184,177,383,157,340,172,170,156,139,370,122,313,131,150,126,123,340,120,296,123,129,126,118,468.0,120.0,317.0,135.0,134.0,134.0,123.0,357.0,155.0,336.0,185.0,172.0,151.0,155.0,395.0,181.0
2,3,0,246,230,555,184,404,224,193,198,211,421,203,412,218,208,194,192,457,224,388,212,202,201,201,481,162,359,174,174,167,159,411,150,353,163,141,179,206,504,...,225,209,209,431,178,390,197,194,181,0,405,136,355,154,155,144,144,381,140,340,148,154,143,143,459.0,142.0,344.0,149.0,153.0,137.0,132.0,416.0,182.0,380.0,194.0,202.0,182.0,186.0,457.0,215.0
3,4,0,221,216,486,157,357,201,199,166,170,370,174,362,180,203,176,164,412,168,341,196,198,175,182,402,122,329,145,167,134,139,333,146,335,156,155,130,166,431,...,173,163,163,367,131,324,149,158,148,131,343,106,304,125,128,114,122,330,103,302,124,127,121,119,441.0,113.0,290.0,130.0,123.0,120.0,113.0,322.0,130.0,323.0,160.0,175.0,141.0,152.0,368.0,184.0
4,5,0,239,248,482,181,324,210,210,182,215,383,166,318,194,219,189,175,392,199,370,195,185,176,172,389,135,279,149,151,154,147,360,146,281,185,152,172,137,397,...,195,175,163,342,130,285,168,157,163,134,322,104,260,129,131,129,107,301,125,263,116,124,118,107,432.0,112.0,271.0,129.0,127.0,120.0,102.0,306.0,140.0,268.0,150.0,158.0,154.0,153.0,370.0,167.0


In [ ]:
items_sales_df.to_csv('/content/drive/My Drive/Grocery/items_sales.csv', index = False)
items_promo_df.to_csv('/content/drive/My Drive/Grocery/items_promo.csv', index = False)
store_sales_df.to_csv('/content/drive/My Drive/Grocery/store_sales.csv', index = False)
store_promo_df.to_csv('/content/drive/My Drive/Grocery/store_promo.csv', index = False)

So with this we have created 6 dataframes which have sales and onpromo information at item-store level, item level, store level...

**Important thing that we saw that we had 200k data points(roughly) on test file, and 167k on train, so do have some data points on test for which we don't have any information on train, also, during baseline model creation the same information was seen, so this is where class information comes handy. We have 4018 items belonging to 337 classes, let us gather same inforamtion that we fetched at store-class level, also, if we don't have a data point where we don't have any information on store-class, we will use class information for such data points, and for remaining, zero prediction as nothing much can be done if we don't have any information with us.**

<h3> Data collection at store-class level </h3>

In [ ]:
store_class_sales_df = train_sales_df
store_class_sales_df['class'] = items_promo_df['class'].values
store_class_sales_df= store_class_sales_df.drop('item_nbr', axis = 1)

In [ ]:
store_class_sales_df = store_class_sales_df.groupby(['class', 'store_nbr']).sum().reset_index()

In [ ]:
store_class_promo_df = final_promo_df
store_class_promo_df['class'] = items_promo_df['class'].values
store_class_promo_df = store_class_promo_df.drop('item_nbr', axis = 1)

In [ ]:
store_class_promo_df = store_class_promo_df.groupby(['class', 'store_nbr']).sum().reset_index()

In [ ]:
store_class_promo_df.head()

,class,store_nbr,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-17,2017-01-18,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,2017-01-25,2017-01-26,2017-01-27,2017-01-28,2017-01-29,2017-01-30,2017-01-31,2017-02-01,2017-02-02,2017-02-03,2017-02-04,2017-02-05,2017-02-06,2017-02-07,...,2017-07-23,2017-07-24,2017-07-25,2017-07-26,2017-07-27,2017-07-28,2017-07-29,2017-07-30,2017-07-31,2017-08-01,2017-08-02,2017-08-03,2017-08-04,2017-08-05,2017-08-06,2017-08-07,2017-08-08,2017-08-09,2017-08-10,2017-08-11,2017-08-12,2017-08-13,2017-08-14,2017-08-15,2017-08-16,2017-08-17,2017-08-18,2017-08-19,2017-08-20,2017-08-21,2017-08-22,2017-08-23,2017-08-24,2017-08-25,2017-08-26,2017-08-27,2017-08-28,2017-08-29,2017-08-30,2017-08-31
0,1002,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,3,3,2,2,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1002,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,3,3,3,5,4,5,4,5,3,2,1,0,1,1,1,1,1,1,1,1,1,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1002,3,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,2,2,3,4,5,5,4,5,0,3,1,1,1,1,1,1,1,1,1,1,1,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1002,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,3,3,5,4,4,3,4,4,2,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
4,1002,5,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,4,3,3,2,4,4,3,5,3,3,0,0,1,1,1,0,1,1,0,1,0,1,0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0


In [ ]:
store_class_sales_df.head()

,class,store_nbr,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-17,2017-01-18,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,2017-01-25,2017-01-26,2017-01-27,2017-01-28,2017-01-29,2017-01-30,2017-01-31,2017-02-01,2017-02-02,2017-02-03,2017-02-04,2017-02-05,2017-02-06,2017-02-07,...,2017-07-07,2017-07-08,2017-07-09,2017-07-10,2017-07-11,2017-07-12,2017-07-13,2017-07-14,2017-07-15,2017-07-16,2017-07-17,2017-07-18,2017-07-19,2017-07-20,2017-07-21,2017-07-22,2017-07-23,2017-07-24,2017-07-25,2017-07-26,2017-07-27,2017-07-28,2017-07-29,2017-07-30,2017-07-31,2017-08-01,2017-08-02,2017-08-03,2017-08-04,2017-08-05,2017-08-06,2017-08-07,2017-08-08,2017-08-09,2017-08-10,2017-08-11,2017-08-12,2017-08-13,2017-08-14,2017-08-15
0,1002,1,0.0,6.291569,11.901285,9.939627,12.817576,10.961278,13.708549,7.454720,18.598319,12.322254,16.385346,13.723816,11.613603,15.181374,8.658693,14.767940,11.443704,16.421714,12.424533,14.909440,19.176284,6.643790,11.731386,14.775909,11.731386,9.862666,16.636661,17.867432,4.787492,14.503975,10.381459,13.053142,10.632774,15.921041,16.203025,7.272398,18.100145,17.264141,...,9.351840,13.235463,8.841014,11.248960,10.632774,14.881269,11.585432,13.746289,13.563968,7.560080,13.117680,11.772208,11.143599,9.149316,12.347572,7.847763,2.079442,8.995165,8.030084,14.999052,19.186106,15.286734,11.248960,6.579251,12.306750,11.836747,13.253813,10.632774,11.164219,8.658693,6.068426,11.966952,13.458607,11.431281,13.367622,5.545177,13.628506,5.375278,18.639141,10.450452
1,1002,2,0.0,27.836761,21.942946,23.265525,20.405583,23.207544,32.629193,33.057327,18.878450,24.403008,22.510759,21.229409,29.223703,27.140769,32.811515,24.109730,20.996215,20.931676,21.896757,22.181838,32.615169,33.486724,22.887637,18.282466,23.880888,18.087494,19.563400,28.226964,30.134961,21.753900,16.624238,22.351737,25.735333,25.198190,28.517311,30.299797,25.950280,18.192854,...,25.229773,22.144098,26.181620,26.004666,15.420266,20.037858,20.082423,16.545676,27.808260,31.840919,22.557279,18.513578,26.574434,17.638109,18.911075,25.241865,27.571552,20.536261,19.253726,22.895605,19.571597,27.772709,31.512428,28.847363,27.442832,24.903783,28.164935,24.597752,24.638574,24.579734,28.459131,25.885741,22.351737,21.326989,16.401095,26.828830,27.567914,25.059788,21.576931,24.121024
2,1002,3,0.0,42.484074,29.286804,35.991684,29.124900,31.628492,36.412191,32.819483,27.527092,26.893368,25.340795,26.562127,30.499108,30.919014,38.440059,32.473441,24.500588,31.257670,25.449504,27.493806,39.247840,39.017547,30.078970,28.135082,24.293734,25.026918,26.248012,35.103897,34.819693,23.118748,31.459318,32.113913,25.907085,29.399255,34.113297,34.262090,28.088892,26.884919,...,31.716228,34.138969,38.759149,31.594623,29.128814,30.958143,27.924841,28.430311,37.213573,40.984867,35.861972,29.948726,30.901972,36.170975,34.766989,35.370941,43.817049,34.242145,27.244687,30.810394,29.437116,33.507269,34.028314,39.867527,34.207775,35.343941,38.776219,31.232261,32.868679,36.456481,38.142670,27.082659,30.859413,28.748667,30.232542,36.638416,31.272846,32.256757,33.608285,24.929109
3,1002,4,0.0,28.353452,21.278199,22.805993,20.207757,19.822911,24.720218,31.524160,21.634874,17.617490,13.523146,16.636661,19.538082,26.514457,29.852150,24.574036,18.456820,15.347359,21.345590,20.009687,26.022354,27.655256,21.108300,21.699412,17.840634,17.174285,16.604036,21.586500,24.419391,19.206725,18.170381,26.406617,24.117277,23.496043,25.139830,30.824783,19.545382,15.392095,...,19.245277,24.359191,26.904792,17.771641,14.767940,16.061297,17.265270,23.000965,26.851303,23.704265,21.642842,20.479691,18.780641,18.123861,17.471536,24.249986,20.883783,21.783428,20.621419,17.106664,14.601886,16.316354,20.292915,25.044039,20.507862,22.184920,20.272296,17.025249,16.348979,17.564246,26.304582,10.961278,17.278515,13.223041,17.735273,19.822911,16.806560,23.187741,16.267563,16.2

In [ ]:
store_class_sales_df.shape

(15826, 229)

In [ ]:
store_class_sales_df.to_csv('/content/drive/My Drive/Grocery/store_class_sales.csv', index = False)
store_class_promo_df.to_csv('/content/drive/My Drive/Grocery/store_class_promo.csv', index = False)

<h2> Data collection using transaction data </h2>

Using the transaction file, only information that I can thought of was the number of items per transaction, which can give us some information, not sure now, how will be using this information, but preparing data using this....

**This df is at store level**

In [ ]:
items_per_transaction = spark.sql('''SELECT transaction_view.store_nbr, transaction_view.date,  total_sales/total_transaction as items_per_transaction
             FROM
             (SELECT store_nbr, date , SUM(transactions) AS total_transaction FROM transactions_data WHERE date > \'2016-12-31\'
             GROUP BY store_nbr, date ORDER BY store_nbr, date) AS transaction_view
             INNER JOIN
             (SELECT store_nbr, date , SUM(unit_sales) AS total_sales FROM data WHERE date > \'2016-12-31\'
             AND unit_sales > 0 GROUP BY store_nbr, date ORDER BY store_nbr, date
             ) AS sales_view
             ON transaction_view.store_nbr = sales_view.store_nbr AND transaction_view.date = sales_view.date 
          ''')

In [ ]:
items_per_transaction.show()

+---------+----------+---------------------+
|store_nbr|      date|items_per_transaction|
+---------+----------+---------------------+
|        1|2017-01-02|    10.89927518148293|
|        1|2017-01-03|    7.673864851875148|
|        1|2017-01-04|    8.586256167065754|
|        1|2017-01-05|    7.077579414771945|
|        1|2017-01-06|    7.201415950817795|
|        1|2017-01-07|    9.036027625511906|
|        1|2017-01-08|    11.01076522496225|
|        1|2017-01-09|    7.960172965873999|
|        1|2017-01-10|    6.963213621073039|
|        1|2017-01-11|    7.575320663429174|
|        1|2017-01-12|    6.419577613287557|
|        1|2017-01-13|   7.3566062226118385|
|        1|2017-01-14|    9.440297620845552|
|        1|2017-01-15|    11.14722532877487|
|        1|2017-01-16|    6.384192097940164|
|        1|2017-01-17|    7.384185392101425|
|        1|2017-01-18|    7.631446812136615|
|        1|2017-01-19|    6.473136167208941|
|        1|2017-01-20|    6.964179811972696|
|        1

In [ ]:
items_per_transaction.cache

<bound method DataFrame.cache of DataFrame[store_nbr: int, date: date, items_per_transaction: double]>

In [ ]:
items_per_transaction = items_per_transaction.groupby('store_nbr').pivot('date').sum('items_per_transaction').fillna(0)

In [ ]:
items_per_transaction = items_per_transaction.toPandas()

In [ ]:
items_per_transaction = items_per_transaction.sort_values('store_nbr').reset_index(drop=True)

In [ ]:
items_per_transaction.head()

,store_nbr,2017-01-01,2017-01-02,2017-01-03,2017-01-04,2017-01-05,2017-01-06,2017-01-07,2017-01-08,2017-01-09,2017-01-10,2017-01-11,2017-01-12,2017-01-13,2017-01-14,2017-01-15,2017-01-16,2017-01-17,2017-01-18,2017-01-19,2017-01-20,2017-01-21,2017-01-22,2017-01-23,2017-01-24,2017-01-25,2017-01-26,2017-01-27,2017-01-28,2017-01-29,2017-01-30,2017-01-31,2017-02-01,2017-02-02,2017-02-03,2017-02-04,2017-02-05,2017-02-06,2017-02-07,2017-02-08,...,2017-07-07,2017-07-08,2017-07-09,2017-07-10,2017-07-11,2017-07-12,2017-07-13,2017-07-14,2017-07-15,2017-07-16,2017-07-17,2017-07-18,2017-07-19,2017-07-20,2017-07-21,2017-07-22,2017-07-23,2017-07-24,2017-07-25,2017-07-26,2017-07-27,2017-07-28,2017-07-29,2017-07-30,2017-07-31,2017-08-01,2017-08-02,2017-08-03,2017-08-04,2017-08-05,2017-08-06,2017-08-07,2017-08-08,2017-08-09,2017-08-10,2017-08-11,2017-08-12,2017-08-13,2017-08-14,2017-08-15
0,1,0.0,10.899275,7.673865,8.586256,7.077579,7.201416,9.036028,11.010765,7.960173,6.963214,7.575321,6.419578,7.356606,9.440298,11.147225,6.384192,7.384185,7.631447,6.473136,6.964180,9.353179,9.822603,7.067403,6.427369,7.360314,6.384721,6.570881,8.928459,10.562131,6.459594,5.893508,7.874516,6.469791,6.667388,9.035299,9.454864,6.124374,6.046076,7.158067,...,6.512046,8.049209,8.862960,6.408477,5.996284,6.967707,6.117179,6.628427,7.377581,8.936624,6.520824,6.911799,7.233911,5.890042,6.805490,8.342479,9.705004,6.139929,5.786803,6.610212,5.829174,6.148100,8.125653,9.537988,6.210857,6.590467,7.678400,5.255284,6.578180,7.504060,8.696742,6.329211,6.268833,7.125511,6.142052,9.480249,7.513569,9.381687,6.020377,5.803043
1,2,0.0,11.458034,9.492144,9.438200,8.255399,8.825503,8.898593,11.483771,9.057660,7.266874,8.677399,6.752764,8.128180,9.449395,11.224434,8.093326,8.369904,8.517010,7.232829,8.964945,9.481430,11.422101,8.823076,7.801587,8.676733,7.092352,7.620987,9.419590,10.455785,7.929985,8.680057,9.042101,7.258740,8.104658,9.411362,10.437884,8.069533,8.124674,8.508163,...,8.342127,8.693160,9.832899,7.544871,6.905922,7.311651,6.675394,7.767705,8.640321,10.676214,7.956902,7.416899,8.098113,7.082144,8.246398,8.578425,9.794374,7.685243,7.198887,7.689183,6.710597,8.308543,9.011161,9.804380,7.727454,8.197805,8.404182,6.956936,7.750873,7.559085,8.918934,7.283796,6.697011,7.063691,6.677188,7.704223,7.676637,8.303270,6.830235,6.445746
2,3,0.0,15.984335,14.377846,14.265220,11.626882,12.636792,13.991900,15.289149,12.328862,11.927903,12.555850,10.939700,11.825405,12.961366,15.119649,12.775539,11.310742,12.835980,10.818190,11.645997,13.488266,15.398105,12.595601,11.342480,12.291217,10.123272,11.827655,12.605720,14.203391,11.607993,11.221293,13.676435,10.077880,11.764752,12.950809,14.809447,11.799860,10.759137,12.966009,...,11.529616,12.527890,13.416589,11.409102,10.949534,11.257657,9.531382,10.266711,12.328329,14.057389,11.818935,10.407421,12.430334,10.058616,10.284237,12.062198,12.874747,11.537438,10.127564,11.798251,9.908683,10.909124,12.197906,13.725640,11.462303,11.305112,12.189158,10.099732,10.561210,11.243110,11.980995,10.529925,10.035571,11.128422,9.164355,10.994905,10.392337,11.331746,10.818084,10.211638
3,4,0.0,13.903885,11.589372,11.222042,9.222110,10.097864,10.429400,13.527836,9.697408,10.155204,10.512618,7.630823,9.479412,9.928692,12.588416,10.178600,8.418148,10.098886,8.016169,9.581724,11.099341,13.166798,10.083997,9.286493,9.831109,7.822068,9.871030,10.042507,13.090830,10.939210,9.077122,10.965539,8.620202,9.687437,10.963957,11.936928,9.471976,8.355161,9.511370,...,8.857806,9.564844,11.652619,8.911685,7.953239,8.310864,7.715328,7.709978,8.644972,11.668931,9.348188,8.777849,9.199172,8.679274,8.878053,9.528542,11.477601,8.972735,8.752778,8.301377,7.651397,8.175107,9.981076,10.980650,8.919753,10.244087,9.544069,7.274645,10.233966,8.723989,11.563005,8.297432,7.987124,8.054134,6.731742,8.316328,8.146044,9.481787,8.491743,7.568267
4,5,0.0,10.072835,8.579036,9.088492,7.200205,8.416129,8.800967,11.340003,8.260114,7.283884,8.508295,7.294558,7.096280,8.245830,10.530729,7.781741,6

In [ ]:
items_per_transaction.shape

(54, 228)

In [ ]:
items_per_transaction.to_csv('/content/drive/My Drive/Grocery/items_per_transaction.csv', index = False)

<h2> Data Collection using holiday file </h2>

Holiday file does have some information, but one thing that striked was the mean sale per store on a holiday day, so what it means is we can find the mean sale per store on a holiday and find the factor that we can multiply with the final result once we have predictions ready, this is one way in which we can actually use information from this file.....
Simplest way is to create vectors from categorical data with information of normal day and holiday day, or create a binary feature if holiday 1 if not 0, then the model will train with this information, but for now collecting data based on the logic of mean sales per store.....

In [ ]:
holidays_events_df.show()

+----------+-------+--------+-------------+--------------------+-----------+
|      date|   type|  locale|  locale_name|         description|transferred|
+----------+-------+--------+-------------+--------------------+-----------+
|2012-03-02|Holiday|   Local|        Manta|  Fundacion de Manta|      False|
|2012-04-01|Holiday|Regional|     Cotopaxi|Provincializacion...|      False|
|2012-04-12|Holiday|   Local|       Cuenca| Fundacion de Cuenca|      False|
|2012-04-14|Holiday|   Local|     Libertad|Cantonizacion de ...|      False|
|2012-04-21|Holiday|   Local|     Riobamba|Cantonizacion de ...|      False|
|2012-05-12|Holiday|   Local|         Puyo|Cantonizacion del...|      False|
|2012-06-23|Holiday|   Local|     Guaranda|Cantonizacion de ...|      False|
|2012-06-25|Holiday|Regional|     Imbabura|Provincializacion...|      False|
|2012-06-25|Holiday|   Local|    Latacunga|Cantonizacion de ...|      False|
|2012-06-25|Holiday|   Local|      Machala|Fundacion de Machala|      False|

In [ ]:
sales_per_store_holiday_type = spark.sql('''SELECT date, type, sum(total_sales)/ count(store_nbr) as sales_per_store FROM
         (SELECT date, store_nbr, type, sum(unit_sales) as total_sales FROM
          (SELECT a.date, a.store_nbr, a.unit_sales, b.type FROM(
            (SELECT date, store_nbr, CASE WHEN unit_sales < 0 THEN 0 ELSE LOG(unit_sales + 1) END as unit_sales FROM data WHERE date >= \'2017-01-01\') a INNER JOIN
              (SELECT date, type FROM holidays_data WHERE date >= \'2017-01-01\' AND date < \'2017-09-01\ AND transferred = FALSE') b ON a.date = b.date))
                GROUP BY date, store_nbr, type ORDER BY date)
                  GROUP BY date, type ORDER BY date
              ''')

In [ ]:
sales_per_store_holiday_type = sales_per_store_holiday_type.groupby('type').pivot('date').sum('sales_per_store').fillna(0).toPandas()

In [ ]:
sales_per_store_holiday_type.head()

,type,2017-01-01,2017-01-02,2017-02-27,2017-02-28,2017-03-02,2017-04-01,2017-04-12,2017-04-13,2017-04-14,2017-04-21,2017-05-01,2017-05-12,2017-05-13,2017-05-14,2017-05-24,2017-05-26,2017-06-23,2017-06-25,2017-07-03,2017-07-23,2017-07-24,2017-07-25,2017-08-05,2017-08-10,2017-08-11,2017-08-15
0,Event,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3447.080891,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Holiday,2507.591043,0.000000,3104.925873,3559.93742,3422.307517,4469.905208,3092.37001,0.000000,6293.927724,3211.195871,4178.683828,3073.583351,0.000000,0.000000,3044.731267,0.000000,3116.962831,11375.599435,7076.320775,3641.852215,0.000000,0.000000,3622.413118,2868.742305,0.000000,3127.507497
2,Transfer,0.000000,4299.217933,0.000000,0.00000,0.000000,0.000000,0.00000,3153.283702,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3608.580404,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3257.485254,0.000000
3,Additional,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,3887.112672,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3255.220525,3024.074115,0.000000,0.000000,0.000000,0.000000


In [ ]:
sales_per_store_holiday_type.to_csv('/content/drive/My Drive/Grocery/sales_per_store_holiday_type.csv', index = False)

In [ ]:
train_sales_df = pd.read_csv('/content/drive/My Drive/Grocery/train_sales.csv')
final_promo_df = pd.read_csv('/content/drive/My Drive/Grocery/final_promo.csv')
items_sales_df = pd.read_csv('/content/drive/My Drive/Grocery/items_sales.csv')
items_promo_df = pd.read_csv('/content/drive/My Drive/Grocery/items_promo.csv')
store_sales_df = pd.read_csv('/content/drive/My Drive/Grocery/store_sales.csv')
store_promo_df = pd.read_csv('/content/drive/My Drive/Grocery/store_promo.csv')
store_class_sales_df = pd.read_csv('/content/drive/My Drive/Grocery/store_class_sales.csv')
store_class_promo_df = pd.read_csv('/content/drive/My Drive/Grocery/store_class_promo.csv')
items_per_transaction = pd.read_csv('/content/drive/My Drive/Grocery/items_per_transaction.csv')
test_df = pd.read_csv('/content/drive/My Drive/Grocery/test.csv')
items_df = pd.read_csv('/content/drive/My Drive/Grocery/items.csv')

In [ ]:
test_df['date'] = pd.to_datetime(test_df['date'])

#Baseline Model

Let us start with building simplest of model as our baseline model.
So, we are doing here is building a model that takes item-store-date information and forecasts result based on 16 week moving average, if we don't have item-store information in test, it uses sales from class-storeand if we don't have even class-store information on our data collection df, it gives results based on the classes that item belong to.

In [ ]:
def generate_baseline_forecast(df, forecast_date):
  '''
  This function takes the df and the forecast_date, calculates the step size
  and based on that generates average with a 16 datapoints window
  '''
  arry = df.values.reshape(-1,1)[-15:]
  step = ( datetime.datetime.date(forecast_date) - datetime.date(2017, 8, 15)).days
  for i in range(step):
    avg = np.mean(arry)
    arry = np.append(arry, avg)
    arry = arry[1:]
    
  return avg

In [ ]:
def moving_average(store, item, forecast_date):
  '''
  This function checks for the level which will be used for moving window average
  '''
  df = item_store_sales_df[(item_store_sales_df['store_nbr'] == store) & (item_store_sales_df['item_nbr'] == item)]
  
  if df.shape[0] == 0:
    return 0
  else:
    return np.expm1(generate_baseline_forecast(df, forecast_date))

In [ ]:
%%time
test_df['unit_sales'] = test_df.apply(lambda x: moving_average(x.store_nbr, x.item_nbr, x.date), axis = 1)

CPU times: user 1h 13min 42s, sys: 2.24 s, total: 1h 13min 45s
Wall time: 1h 13min 45s


In [ ]:
test_df.head()

,id,date,store_nbr,item_nbr,onpromotion,unit_sales
0,125497040,2017-08-16,1,96995,False,0.366349
1,125497041,2017-08-16,1,99197,False,0.212509
2,125497042,2017-08-16,1,103501,False,0.000000
3,125497043,2017-08-16,1,103520,False,0.873407
4,125497044,2017-08-16,1,103665,False,1.915161


In [ ]:
test_df[test_df.isnull().any(axis = 1)]['item_nbr'].unique()

array([], dtype=int64)

In [ ]:
#Creating a submission file for kaggle submission to get a score based on the baseline model
test_df[['id', 'unit_sales']].to_csv('baseline_16_submission.csv', index = False)

After making submission from the results generated on test using the baseline model, score was .59249 on private, this score we will consider as a base score for other models, we need to create models that will improve the score than this.....

#Data Preparation

**imports**

In [5]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
from datetime import timedelta
import calendar
import math
import time
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
import xgboost as xgb
from joblib import dump, load
from sklearn import preprocessing
import category_encoders as ce

**Loading files**

In [6]:
item_store_sales_df = pd.read_csv('/content/drive/My Drive/Grocery/train_sales.csv')
item_store_promo_df = pd.read_csv('/content/drive/My Drive/Grocery/final_promo.csv')
#items_sales_df = pd.read_csv('/home/jupyter/final_files/items_sales.csv')
#items_promo_df = pd.read_csv('/home/jupyter/final_files/items_promo.csv')
#store_sales_df = pd.read_csv('/home/jupyter/final_files/store_sales.csv')
#store_promo_df = pd.read_csv('/home/jupyter/final_files/store_promo.csv')
#store_class_sales_df = pd.read_csv('/home/jupyter/final_files/store_class_sales.csv')
#store_class_promo_df = pd.read_csv('/home/jupyter/final_files/store_class_promo.csv')
#items_per_transaction_df = pd.read_csv('/home/jupyter/final_files/items_per_transaction.csv')
test_df = pd.read_csv('/content/drive/My Drive/Grocery/test.csv')
items_df = pd.read_csv('/content/drive/My Drive/Grocery/items.csv')
stores_df = pd.read_csv('/content/drive/My Drive/Grocery/stores.csv')

The data collected at item, store, store-class level was not used as it was adding noise to all the models, and using them score got impacted, even transaction data was not used for final modelling.

**Approcah to solve the problem:**
1. First point that we need to understand is how our test file looks like, so if we take dates in columns we will see that for an item-store combination, we need to make prediction for date from 16th Aug till 31st Aug, so we can say that we need to make prediction from t+1 till t+16 time stamp using the date we have.....
2. Let us collect X_i such that the next 16 dates can be thought of as our Y_i, so we can use historical data to generate X_i & next 16 intervals as Y_i(unit_sales), train our model using this and make predictions on test.
3. How to use this, let say X_i is a matrix of [m x n] & Y_i is a matrix of [m x 16], for each Y_i or the step that Y_i represents, train using X_i, i.e, at each iteration we will use X_i[m x n] and Y_i[m x 1] and make prediction using the same, so, for 1st set of Y_i, we will use X_i and get step1 forecast, for 2nd set of Y_i, we will again use the same X_i and get step2 forecast, and so on till 16th step. Our resultant prediction will also be a vector of [m x 16], where m is the number of data points(item-store level), and 16 are the date ranging from 16th Aug till 31st Aug.

<h2> Generating df for categorical features </h2>

In [7]:
stores_df.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [9]:
class_family_df = pd.DataFrame(item_store_sales_df['item_nbr']).merge(items_df[['item_nbr', 'class', 'family', 'perishable']], on = 'item_nbr', how = 'left')
class_family_df['class'] = class_family_df['class'].astype('str')
class_family_df['item_nbr'] = class_family_df['item_nbr'].astype('str')
class_family_df.head()

,item_nbr,class,family,perishable
0,96995,1093,GROCERY I,0
1,99197,1067,GROCERY I,0
2,103520,1028,GROCERY I,0
3,103665,2712,BREAD/BAKERY,1
4,105574,1045,GROCERY I,0


In [10]:
store_detail_df = pd.DataFrame(item_store_sales_df['store_nbr']).merge(stores_df[['store_nbr', 'state', 'city', 'type', 'cluster']], on = 'store_nbr', how = 'left')
store_detail_df['store_nbr'] = store_detail_df['store_nbr'].astype('str')
store_detail_df['cluster'] = store_detail_df['cluster'].astype('str')
store_detail_df.head()

,store_nbr,state,city,type,cluster
0,1,Pichincha,Quito,D,13
1,1,Pichincha,Quito,D,13
2,1,Pichincha,Quito,D,13
3,1,Pichincha,Quito,D,13
4,1,Pichincha,Quito,D,13


Y_train:

Y_cv:

Y_test: 

<h1> Helper Functions </h1>

In [11]:
def cat_encoding(cat_data, category):
  '''
  This function takes a df and the category and generate
  binary encoded vectors for the same
  '''
  encoder = ce.BinaryEncoder()
  return encoder.fit_transform(cat_data[category]).values

In [12]:
#Generating binary encoded vector for categories part of item table
class_array = cat_encoding(class_family_df, 'class')
family_array = cat_encoding(class_family_df, 'family')
item_array = cat_encoding(class_family_df, 'item_nbr')

In [13]:
print(class_array.shape, family_array.shape, item_array.shape)

(167515, 10) (167515, 7) (167515, 13)


In [14]:
store_detail_df.head()

,store_nbr,state,city,type,cluster
0,1,Pichincha,Quito,D,13
1,1,Pichincha,Quito,D,13
2,1,Pichincha,Quito,D,13
3,1,Pichincha,Quito,D,13
4,1,Pichincha,Quito,D,13


In [15]:
# Generating binary encoded vectors for category part of store table
store_array = cat_encoding(store_detail_df, 'store_nbr')
store_state_array = cat_encoding(store_detail_df, 'state')
store_city_array = cat_encoding(store_detail_df, 'city')
store_type_array = cat_encoding(store_detail_df, 'type')
store_cluster_array = cat_encoding(store_detail_df, 'cluster')

In [16]:
print(store_array.shape, store_state_array.shape, store_city_array.shape, store_type_array.shape, store_cluster_array.shape)

(167515, 7) (167515, 5) (167515, 6) (167515, 4) (167515, 6)


In [17]:
store_array

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 1, 1, ..., 1, 1, 0],
       [0, 1, 1, ..., 1, 1, 0],
       [0, 1, 1, ..., 1, 1, 0]])

In [18]:
def get_data(data, dt_end, days, period, freq='D'):
  '''
  This function gives us the selected columns based on a range of dates passed.
  '''
  return data[[str(col)[0:10] for col in pd.date_range(dt_end - datetime.timedelta(days = days), periods = period, freq = freq)]]

In [19]:
def average(data):
  '''
  Here we are calculating simple average
  '''
  return np.mean(data, axis = 1)

In [20]:
def weighted_moving_average(data):
  '''
  This function computes weighted moving average, 
  higher weights are given to recent observations.
  '''
  data = data.values
  weight_len = data.shape[1]
  denom = (weight_len *(weight_len + 1))/2
  weights = [i+1/denom for i in range(weight_len)]
  data = average(data * weights)
  return data

In [21]:
#This was excluded from the final features, as this was of no use to the models.
def expo_smoothing(data_row, alpha = 0.7):
  '''
  This function gives us the exponential smoothing compoenent of our time series.
  '''
  values = [data_row[0]]
  for i in range(len(data_row)):
    values.append(alpha * data_row[i] + (1 - alpha) * values[i - 1])
  return values

In [22]:
#These functions were excluded from the model, here we are calculating 
#triple exponential smoothing, also known as Holt's Winter technique.
#Ref: https://www.itl.nist.gov/div898/handbook/pmc/section4/pmc435.htm
def trend_component(data_row, season_len):
  sum = 0
  for i in range(season_len):
    sum += (data_row[i + season_len] - data_row[i])/season_len
  return sum/season_len

def seasonal_components(data_row, season_len):
  n_seasons = int(len(data_row)/season_len) #Total number of seasons in our series
  #next we find the average value of each season, let say if we have 70 data points with 7 as our season lenght, we 
  # have total 10 season, so, for each of these seasons we find the average value
  average = [sum(data_row[i*season_len:(i*season_len) + season_len])/season_len for i in range(n_seasons)]
  #print(average)
  #The computed average will be subtracted from the appropriate season and we will take corresponding point of each season to find the final seasonal component

  dict_season = {i: sum([data_row[season_len * j + i] - average[j] for j in range(n_seasons)])/n_seasons for i in range(season_len) }
  #print(dict_season)
  return dict_season

def triple_expo_smoothing(data_row, season_len = 7, alpha = 0.7 , beta = 0.4, gamma = 0.2):
  result = []
  trend = trend_component(data_row, season_len) #Initial Trend
  seasonal_component = seasonal_components(data_row, season_len) #Initial Seasonal component
  #print(seasonal_component)
  for i in range(len(data_row)):
    if i == 0:
      smooth = data_row[0]
      result.append(data_row[0])
      continue
    value = data_row[i]
    pre_smooth, smooth = smooth, alpha*(value - seasonal_component[i % season_len]) + (1 - alpha)*(smooth + trend) #Overall Smoothing
    trend = beta * (smooth - pre_smooth) + (1- beta) * trend #Trend Smoothing
    seasonal_component[i % season_len] = gamma * (value - smooth) + (1 - gamma) * seasonal_component[i % season_len] #Seasonal Smoothing
    result.append(smooth + trend + seasonal_component[i % season_len])
  return result

In [23]:
def feature_engg_sales(data, end_date, prefix):
  '''
  This function generates feature dictionary for train, cv, test
  Features generated are:
  moving average, weighted moving average, standard deviation observed, 
  moving average of DOW, weighted moving average of DOW, having total sales day,
  last sales day in n days, first sales day in n days
  '''
  days_list = [3, 7, 16, 30, 60, 120] # These are the list of days used for extracting above mentioned features 
  #feature_dict = {}
  feature_dict = {'{}_average_{}_days'.format(prefix, days): average(get_data(data, end_date, days, days).values)  for days in days_list}
  feature_dict.update({'{}_WMA_{}_days'.format(prefix, days): weighted_moving_average(get_data(data, end_date, days, days)) for days in days_list})
  #feature_dict.update({'{}_average_diff_{}_days'.format(prefix, days) : get_data(data, end_date, days, days).diff(axis =1).mean(axis = 1).values for days in days_list})
  #feature_dict.update({'{}_max_{}_days'.format(prefix, days) : get_data(data, end_date, days, days).max(axis = 1).values for days in days_list})
  feature_dict.update({'{}_std_{}_days'.format(prefix, days) : get_data(data, end_date, days, days).std(axis = 1).values for days in days_list})
  feature_dict.update({'{}_6avgdow_{}_days'.format(prefix, day) : get_data(data, end_date, 42 - day, 6, freq = '7D').mean(axis =1).values for day in range(7)})
  feature_dict.update({'{}_20avgdow_{}_days'.format(prefix, day) : get_data(data, end_date, 140 - day, 20, freq = '7D').mean(axis =1).values for day in range(7)})
  feature_dict.update({'{}_6WMAdow_{}_days'.format(prefix, day) : weighted_moving_average(get_data(data, end_date, 42 - day, 6, freq = '7D')) for day in range(7)})
  feature_dict.update({'{}_20WMAdow_{}_days'.format(prefix, day) : weighted_moving_average(get_data(data, end_date, 140 - day, 20, freq = '7D')) for day in range(7)})
  feature_dict.update({'{}_has_sale_day_{}'.format(prefix, days) : (get_data(data, end_date, days, days) > 0).sum(axis = 1).values for days in days_list})
  feature_dict.update({'{}_last_has_sale_day_{}'.format(prefix, days) : days - ((get_data(data, end_date, days, days) > 0) * np.arange(days)).max(axis = 1).values for days in days_list})
  feature_dict.update({'{}_first_has_sale_day_{}'.format(prefix, days) : ((get_data(data, end_date, days, days) > 0) * np.arange(days, 0, -1)).max(axis = 1).values for days in days_list})


  #feature_dict.update({'{}_lastday'.format(prefix) : get_data(data, end_date, 1, 1).values.ravel()})
  #feature_dict.update({'{}_day_{}'.format(prefix, day) : get_data(data, end_date, day, 1).values.ravel() for day in range(1, 16)})

  #exponential smoothing: smoothing 16 days data point with a smoothing factor of 0.7
  #df = get_data(data, end_date, 16, 16)
  #expo_arry = np.array([expo_smoothing(df.iloc[i])[1:] for i in range(df.shape[0])])
  #feature_dict.update({'expo_smooth_{}'.format(col_num): expo_arry[:, col_num] for col_num in range(expo_arry.shape[1])})

  #Triple Exponential Smoothing(Holt's Winter)
  #df = get_data(data, end_date, 35, 35)
  #holt_winter_arry = np.array([triple_expo_smoothing(df.iloc[i]) for i in range(df.shape[0])])
  #feature_dict.update({'holt_winter_{}'.format(col_num): holt_winter_arry[:, col_num] for col_num in range(holt_winter_arry.shape[1])})

  return feature_dict

In [24]:
def feature_engg_promo(data, class_array, store_array, end_date, prefix):
    '''
    This function uses promo information and categorical array to create features
    features created are---
    promo: total_promo, future promo information, promo days in 15 days, last promo in 15 days, first promo in 15 days
    categorical: class, item, store, family, city, state, clsuter, type 
    '''
    days_list = [16, 30, 60, 120]
    feature_dict = {'{}_totalpromo_{}_days'.format(prefix, days) : get_data(data, end_date, days, days).sum(axis = 1).values for days in days_list}
    feature_dict.update({'{}_totalpromoafter_{}_days'.format(prefix, days) : get_data(data, end_date + timedelta(days = 16), 16, days).sum(axis = 1).values for days in [5, 10, 15]})
   # if prefix in ['item', 'store_class']:
   #     feature_dict.update({'{}_maxnopromo_{}_days'.format(prefix, days) : get_data(data, end_date, days, days).max(axis = 1).values for days in days_list})
   #     feature_dict.update({'{}_maxnopromoafter_{}_days'.format(prefix, days) : get_data(data, end_date + timedelta(days = 16), 16, days).max(axis = 1).values for days in [5, 10, 15]})
    feature_dict.update({'{}_promo_{}_day'.format(prefix, abs(day - 1)): get_data(data, end_date, day, 1).values.ravel() for day in range(-15, 1)})
    feature_dict.update({'promo_day_in_15_days' : (get_data(data, end_date + timedelta(days=16), 15, 15) > 0).sum(axis = 1).values})
    feature_dict.update({'last_promo_day_in_15_days' : 15 - ((get_data(data, end_date + timedelta(days=16), 15, 15) > 0) * np.arange(15)).max(axis = 1).values})
    feature_dict.update({'firt_promo_day_in_15_days' : ((get_data(data, end_date + timedelta(days=16), 15, 15) > 0) * np.arange(15, 0, -1)).max(axis = 1).values})
    feature_dict.update({'class_{}'.format(i+1) : class_array[:, i] for i in range(class_array.shape[1])})
    feature_dict.update({'item_{}'.format(i+1) : item_array[:, i] for i in range(item_array.shape[1])})
    feature_dict.update({'store_{}'.format(i+1) : store_array[:, i] for i in range(store_array.shape[1])})
    feature_dict.update({'family_{}'.format(i+1) : family_array[:, i] for i in range(family_array.shape[1])})
    feature_dict.update({'city_{}'.format(i+1) : store_city_array[:, i] for i in range(store_city_array.shape[1])})
    feature_dict.update({'state_{}'.format(i+1) : store_state_array[:, i] for i in range(store_state_array.shape[1])})
    feature_dict.update({'cluster_{}'.format(i+1) : store_cluster_array[:, i] for i in range(store_cluster_array.shape[1])})
    feature_dict.update({'type_{}'.format(i+1) : store_type_array[:, i] for i in range(store_type_array.shape[1])})# not needed
    feature_dict.update({'perishable' : class_family_df['perishable'].values})
    #feature_dict.update({'class_{}'.format(i + 1) : class_vector.toarray()[:, i] for i in range(class_vector.shape[1])})
    #feature_dict.update({'{}_promo_{}_day'.format(prefix, day - 1): get_data(data, end_date, day, 1).values.ravel() for day in range(-15, 15)})
    
    return feature_dict

<h1> Preparing Train Data </h1>

In [25]:
#To create training points we will take multiple intervals and will concat all the information we got from these intervals....
x_lst, y_lst = [], []
num_of_intervals = 8
dates = [date(2017, 5, 31) + timedelta(days=7 * interval) for interval in range(num_of_intervals)]
for train_date in tqdm(dates):
  train_dict = feature_engg_sales(item_store_sales_df, train_date,'item_store')
  x_lst.append(pd.DataFrame(train_dict, index = [i for i in range(len(list(train_dict.values())[0]))]))
  y_lst.append(item_store_sales_df[[str(col)[0:10] for col in pd.date_range(train_date, periods = 16)]].values)

train_item_store_x = pd.concat(x_lst, axis=0)
train_y = np.concatenate(y_lst, axis=0)
del x_lst, y_lst
print(train_item_store_x.shape, train_y.shape)

100%|██████████| 8/8 [00:18<00:00,  2.34s/it]


(1340120, 64) (1340120, 16)


In [26]:
train_item_store_x.head()

,item_store_average_3_days,item_store_average_7_days,item_store_average_16_days,item_store_average_30_days,item_store_average_60_days,item_store_average_120_days,item_store_WMA_3_days,item_store_WMA_7_days,item_store_WMA_16_days,item_store_WMA_30_days,item_store_WMA_60_days,item_store_WMA_120_days,item_store_std_3_days,item_store_std_7_days,item_store_std_16_days,item_store_std_30_days,item_store_std_60_days,item_store_std_120_days,item_store_6avgdow_0_days,item_store_6avgdow_1_days,item_store_6avgdow_2_days,item_store_6avgdow_3_days,item_store_6avgdow_4_days,item_store_6avgdow_5_days,item_store_6avgdow_6_days,item_store_20avgdow_0_days,item_store_20avgdow_1_days,item_store_20avgdow_2_days,item_store_20avgdow_3_days,item_store_20avgdow_4_days,item_store_20avgdow_5_days,item_store_20avgdow_6_days,item_store_6WMAdow_0_days,item_store_6WMAdow_1_days,item_store_6WMAdow_2_days,item_store_6WMAdow_3_days,item_store_6WMAdow_4_days,item_store_6WMAdow_5_days,item_store_6WMAdow_6_days,item_store_20WMAdow_0_days,item_store_20WMAdow_1_days,item_store_20WMAdow_2_days,item_store_20WMAdow_3_days,item_store_20WMAdow_4_days,item_store_20WMAdow_5_days,item_store_20WMAdow_6_days,item_store_has_sale_day_3,item_store_has_sale_day_7,item_store_has_sale_day_16,item_store_has_sale_day_30,item_store_has_sale_day_60,item_store_has_sale_day_120,item_store_last_has_sale_day_3,item_store_last_has_sale_day_7,item_store_last_has_sale_day_16,item_store_last_has_sale_day_30,item_store_last_has_sale_day_60,item_store_last_has_sale_day_120,item_store_first_has_sale_day_3,item_store_first_has_sale_day_7,item_store_first_has_sale_day_16,item_store_first_has_sale_day_30,item_store_first_has_sale_day_60,item_store_first_has_sale_day_120
0,0.231049,0.297063,0.129965,0.138629,0.163697,0.081849,0.038508,0.802778,1.517215,2.380103,5.016269,7.419023,0.400189,0.370503,0.279417,0.281999,0.345724,0.256935,0.115525,0.115525,0.414151,0.346574,0.115525,0.115525,0.115525,0.034657,0.034657,0.179176,0.103972,0.034657,0.069315,0.034657,0.121026,0.583124,0.481820,0.594126,0.583124,0.236550,0.236550,0.520025,0.658655,2.538085,1.629391,0.658655,1.005393,0.554683,1,3,3,6,12,12,3,3,3,3,3,3,3,6,6,23,54,54
1,0.597253,0.610952,0.585266,0.381457,0.190728,0.104519,0.330591,1.342745,3.598441,6.980830,9.211961,10.346105,0.555548,0.605987,0.556407,0.498290,0.398793,0.310351,0.481729,0.462098,0.298627,0.000000,0.183102,0.298627,0.183102,0.144519,0.248491,0.089588,0.158903,0.158903,0.089588,0.054931,2.017431,1.985922,1.093202,0.000000,0.924229,1.141149,0.558025,2.622296,2.750893,1.578353,0.139386,1.148410,1.592737,0.934082,2,4,9,12,12,13,2,2,2,2,2,2,3,7,16,21,21,118
2,0.000000,0.824046,0.728402,0.868856,0.822436,0.737244,0.000000,1.118661,5.109683,10.959512,24.484038,47.968203,0.000000,0.872092,0.696327,0.732264,0.677539,0.708917,0.883053,1.327591,0.969519,1.126929,0.115525,0.797915,0.865493,0.857374,0.953570,0.793105,0.657925,0.213833,0.774370,0.736356,2.612875,3.411548,2.010896,2.721672,0.236550,1.328396,2.108017,8.821638,11.261035,7.718030,7.303267,1.586762,7.894452,7.994472,0,4,10,21,41,75,3,4,4,4,4,4,0,7,16,30,60,120
3,0.366204,0.709973,0.939201,0.956552,0.995797,1.011112,0.427238,2.056255,6.291940,13.607748,28.485405,58.984437,0.634284,0.784219,0.781861,0.707439,0.641178,0.662595,0.597253,1.260013,1.316835,1.636974,0.231049,0.732408,1.096542,0.954670,1.038369,1.408288,1.412923,0.453208,0.850698,0.989327,0.625694,2.898285,3.409153,4.256465,0.357576,2.598305,2.794606,8.083967,10.101371,12.348602,14.095943,4.415203,7.718346,9.742277,1,4,11,22,47,93,2,2,2,2,2,2,2,6,16,29,60,120
4,1.059351,1.403121,1.648731,1.774012,1.828190,1.818254,1.371065,3.863389,11.002819,24.010000,51.428602,107.818523,0.348237,0.448346,0.749072,0.692988,0.684553,0.655836,2.039207,1.849568,2.038974,1.536385,0.828302,2.085004,1.924844,2.161346,2.022376,1.900244,1.626346,0.840328,2.042486,2.044028,5.505694,4.494860,5.045201,4.024805,1.811572,4.943188,4.830423,20.555658,18.649828,18.724818,15.355110,8.204936,20.070135,19.289

In [27]:
x_lst = []
num_of_intervals = 8
dates = [date(2017, 5, 31) + timedelta(days=7 * interval) for interval in range(num_of_intervals)]
for train_date in tqdm(dates):
  train_dict = feature_engg_promo(item_store_promo_df, class_array, store_array, train_date,'item_store')
  x_lst.append(pd.DataFrame(train_dict, index = [i for i in range(len(list(train_dict.values())[0]))]))

train_item_store_x1 = pd.concat(x_lst, axis=0)
del x_lst
print(train_item_store_x1.shape)

100%|██████████| 8/8 [00:02<00:00,  2.74it/s]


(1340120, 85)


In [28]:
train_item_store_x1.head()

,item_store_totalpromo_16_days,item_store_totalpromo_30_days,item_store_totalpromo_60_days,item_store_totalpromo_120_days,item_store_totalpromoafter_5_days,item_store_totalpromoafter_10_days,item_store_totalpromoafter_15_days,item_store_promo_16_day,item_store_promo_15_day,item_store_promo_14_day,item_store_promo_13_day,item_store_promo_12_day,item_store_promo_11_day,item_store_promo_10_day,item_store_promo_9_day,item_store_promo_8_day,item_store_promo_7_day,item_store_promo_6_day,item_store_promo_5_day,item_store_promo_4_day,item_store_promo_3_day,item_store_promo_2_day,item_store_promo_1_day,promo_day_in_15_days,last_promo_day_in_15_days,firt_promo_day_in_15_days,class_1,class_2,class_3,class_4,class_5,class_6,class_7,class_8,class_9,class_10,item_1,item_2,item_3,item_4,...,item_10,item_11,item_12,item_13,store_1,store_2,store_3,store_4,store_5,store_6,store_7,family_1,family_2,family_3,family_4,family_5,family_6,family_7,city_1,city_2,city_3,city_4,city_5,city_6,state_1,state_2,state_3,state_4,state_5,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,type_1,type_2,type_3,type_4,perishable
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1
4,0,0,21,34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0


In [29]:
train_x = train_item_store_x.reset_index(drop = True).merge(train_item_store_x1.reset_index(drop = True), left_index=True, right_index=True)

In [30]:
[train_x[col].update((train_x[col] - train_x[col].min()) / (train_x[col].max() - train_x[col].min())) for col in train_x.columns]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [31]:
train_x.head()

,item_store_average_3_days,item_store_average_7_days,item_store_average_16_days,item_store_average_30_days,item_store_average_60_days,item_store_average_120_days,item_store_WMA_3_days,item_store_WMA_7_days,item_store_WMA_16_days,item_store_WMA_30_days,item_store_WMA_60_days,item_store_WMA_120_days,item_store_std_3_days,item_store_std_7_days,item_store_std_16_days,item_store_std_30_days,item_store_std_60_days,item_store_std_120_days,item_store_6avgdow_0_days,item_store_6avgdow_1_days,item_store_6avgdow_2_days,item_store_6avgdow_3_days,item_store_6avgdow_4_days,item_store_6avgdow_5_days,item_store_6avgdow_6_days,item_store_20avgdow_0_days,item_store_20avgdow_1_days,item_store_20avgdow_2_days,item_store_20avgdow_3_days,item_store_20avgdow_4_days,item_store_20avgdow_5_days,item_store_20avgdow_6_days,item_store_6WMAdow_0_days,item_store_6WMAdow_1_days,item_store_6WMAdow_2_days,item_store_6WMAdow_3_days,item_store_6WMAdow_4_days,item_store_6WMAdow_5_days,item_store_6WMAdow_6_days,item_store_20WMAdow_0_days,...,item_10,item_11,item_12,item_13,store_1,store_2,store_3,store_4,store_5,store_6,store_7,family_1,family_2,family_3,family_4,family_5,family_6,family_7,city_1,city_2,city_3,city_4,city_5,city_6,state_1,state_2,state_3,state_4,state_5,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,type_1,type_2,type_3,type_4,perishable
0,0.029921,0.043809,0.019646,0.021161,0.025172,0.012648,0.004012,0.037333,0.029881,0.024803,0.026159,0.019269,0.079716,0.087522,0.071441,0.073215,0.097776,0.082831,0.016763,0.018608,0.051552,0.050391,0.014264,0.015660,0.018146,0.005073,0.005638,0.026422,0.015494,0.005194,0.009412,0.005474,0.006870,0.036934,0.024047,0.033467,0.027582,0.012560,0.014026,0.008001,...,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0
1,0.077346,0.090100,0.088472,0.058228,0.029329,0.016152,0.034439,0.062443,0.070871,0.072748,0.048039,0.026872,0.110663,0.143150,0.142262,0.129370,0.112785,0.100051,0.069900,0.074430,0.037172,0.000000,0.022608,0.040480,0.028761,0.021155,0.040428,0.013211,0.023680,0.023813,0.012165,0.008677,0.114517,0.125784,0.054561,0.000000,0.043717,0.060592,0.033088,0.040348,...,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0
2,0.000000,0.121526,0.110109,0.132627,0.126468,0.113929,0.000000,0.052022,0.100634,0.114211,0.127681,0.124587,0.000000,0.206011,0.178037,0.190117,0.191619,0.228541,0.128133,0.213836,0.120683,0.163853,0.014264,0.108161,0.135950,0.125504,0.155139,0.116956,0.098045,0.032044,0.105151,0.116315,0.148317,0.216080,0.100362,0.153311,0.011189,0.070534,0.124995,0.135735,...,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0
3,0.047424,0.104703,0.141975,0.146014,0.153126,0.156251,0.044507,0.095625,0.123919,0.141809,0.148548,0.153199,0.126347,0.185253,0.199907,0.183671,0.181336,0.213607,0.086663,0.202951,0.163916,0.238012,0.028528,0.099281,0.172243,0.139746,0.168935,0.207674,0.210556,0.067916,0.115515,0.156274,0.035517,0.183571,0.170147,0.239765,0.016914,0.137963,0.165706,0.124385,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1
4,0.137189,0.206924,0.249231,0.270796,0.281126,0.280982,0.142830,0.179664,0.216699,0.250212,0.268194,0.280035,0.069368,0.105911,0.191523,0.179920,0.193603,0.211428,0.295893,0.297911,0.253806,0.223387,0.102271,0.282632,0.302351,0.316381,0.329025,0.280221,0.242361,0.125929,0.277347,0.322874,0.312525,0.284695,0.251801,0.226716,0.085689,0.262470,0.286420,0.316281,...,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0


In [32]:
print('Shape of train_x and corresponding train_y is {} & {}'.format(train_x.shape, train_y.shape))

Shape of train_x and corresponding train_y is (1340120, 149) & (1340120, 16)


<h1> Preparing CV Data </h1>

In [34]:
#Generating sales features
cv_date = date(2017, 7, 26)
cv_dict = feature_engg_sales(item_store_sales_df, cv_date, 'item_store')
cv_item_store_x = pd.DataFrame(cv_dict, index = [i for i in range(len(list(cv_dict.values())[0]))])
cv_item_store_x.shape

(167515, 64)

In [35]:
cv_item_store_x.head()

,item_store_average_3_days,item_store_average_7_days,item_store_average_16_days,item_store_average_30_days,item_store_average_60_days,item_store_average_120_days,item_store_WMA_3_days,item_store_WMA_7_days,item_store_WMA_16_days,item_store_WMA_30_days,item_store_WMA_60_days,item_store_WMA_120_days,item_store_std_3_days,item_store_std_7_days,item_store_std_16_days,item_store_std_30_days,item_store_std_60_days,item_store_std_120_days,item_store_6avgdow_0_days,item_store_6avgdow_1_days,item_store_6avgdow_2_days,item_store_6avgdow_3_days,item_store_6avgdow_4_days,item_store_6avgdow_5_days,item_store_6avgdow_6_days,item_store_20avgdow_0_days,item_store_20avgdow_1_days,item_store_20avgdow_2_days,item_store_20avgdow_3_days,item_store_20avgdow_4_days,item_store_20avgdow_5_days,item_store_20avgdow_6_days,item_store_6WMAdow_0_days,item_store_6WMAdow_1_days,item_store_6WMAdow_2_days,item_store_6WMAdow_3_days,item_store_6WMAdow_4_days,item_store_6WMAdow_5_days,item_store_6WMAdow_6_days,item_store_20WMAdow_0_days,item_store_20WMAdow_1_days,item_store_20WMAdow_2_days,item_store_20WMAdow_3_days,item_store_20WMAdow_4_days,item_store_20WMAdow_5_days,item_store_20WMAdow_6_days,item_store_has_sale_day_3,item_store_has_sale_day_7,item_store_has_sale_day_16,item_store_has_sale_day_30,item_store_has_sale_day_60,item_store_has_sale_day_120,item_store_last_has_sale_day_3,item_store_last_has_sale_day_7,item_store_last_has_sale_day_16,item_store_last_has_sale_day_30,item_store_last_has_sale_day_60,item_store_last_has_sale_day_120,item_store_first_has_sale_day_3,item_store_first_has_sale_day_7,item_store_first_has_sale_day_16,item_store_first_has_sale_day_30,item_store_first_has_sale_day_60,item_store_first_has_sale_day_120
0,0.000000,0.354987,0.155307,0.105935,0.122282,0.131437,0.000000,0.879596,1.778179,2.176918,3.428349,7.519188,0.000000,0.611917,0.427617,0.335926,0.307939,0.319107,0.115525,0.115525,0.231049,0.183102,0.115525,0.000000,0.000000,0.103972,0.069315,0.283148,0.158903,0.069315,0.069315,0.034657,0.005501,0.236550,1.166248,0.924229,0.121026,0.000000,0.000000,1.144188,0.936079,2.872698,1.841558,0.901421,0.450876,0.277424,0,2,2,3,9,19,3,4,4,4,4,4,0,5,5,27,60,110
1,0.000000,0.610952,0.664548,0.598990,0.657984,0.409425,0.000000,0.929835,4.674705,8.861481,18.192273,32.906322,0.000000,0.678898,0.607217,0.594421,0.638202,0.578104,0.943827,0.645200,0.645200,0.597253,0.115525,0.298627,1.017466,0.462324,0.436162,0.372736,0.356943,0.144519,0.289037,0.394828,2.270485,1.532543,2.794627,1.704676,0.352075,0.476318,1.550270,6.515284,5.865737,5.511722,5.226714,1.853267,3.643109,5.670702,0,4,10,17,36,46,3,4,4,4,4,4,0,7,15,29,59,77
2,1.059351,0.850092,0.804148,0.788443,0.810735,0.849512,1.371065,3.051470,6.273850,11.680720,24.097840,49.630231,0.348237,0.488764,0.590404,0.590111,0.574882,0.632085,1.017466,1.132990,1.085043,0.876249,0.298627,0.760725,0.549306,1.012206,1.244426,0.930947,0.854712,0.193560,0.947657,0.808333,2.012368,2.990824,2.639342,2.487372,0.774945,2.154928,1.674076,9.386022,11.145653,9.449900,8.748309,2.072546,7.737940,7.532716,3,6,12,22,45,89,1,1,1,1,1,1,3,7,15,29,60,120
3,1.229626,0.881969,0.902465,0.901965,1.040801,1.025935,1.665613,3.236268,6.662618,13.413959,28.991042,60.421976,0.477814,0.667860,0.727763,0.814505,0.745399,0.685840,1.096542,0.876249,1.313467,1.547884,0.529676,0.730338,0.895880,1.139114,0.900527,1.304937,1.477980,0.612111,0.816828,0.996004,2.860113,1.746278,2.840830,3.352578,1.016996,2.069123,2.941719,9.906517,7.832664,12.183876,14.801181,5.235038,8.144763,9.836275,3,5,11,19,45,93,1,1,1,1,1,1,3,7,16,27,60,119
4,1.866141,1.892588,1.820677,1.739691,1.686863,1.792781,2.237938,5.693231,13.877289,26.037052,51.067484,102.946858,0.300870,0.340875,0.333653,0.542163,0.588243,0.637878,2.049380,1.674165,1.936412,1.525628,0.797915,2.156327,1.787064,2.106680,1.911962,2.009095,1.600804,0.854712,2.137850,1.966089,4.952796,5.047465,4.879910,4.156307,2.689225,5.284227,4.684755,19.636048,17.407744,18.433788,15.063932,8.264486,19.914719,17.496

In [36]:
#Generating promo and categorical features
cv_dict = feature_engg_promo(item_store_promo_df, class_array, store_array, cv_date, 'item_store')
cv_item_store_x1 = pd.DataFrame(cv_dict, index = [i for i in range(len(list(cv_dict.values())[0]))])
cv_item_store_x1.shape

(167515, 85)

In [37]:
cv_item_store_x1.head()

,item_store_totalpromo_16_days,item_store_totalpromo_30_days,item_store_totalpromo_60_days,item_store_totalpromo_120_days,item_store_totalpromoafter_5_days,item_store_totalpromoafter_10_days,item_store_totalpromoafter_15_days,item_store_promo_16_day,item_store_promo_15_day,item_store_promo_14_day,item_store_promo_13_day,item_store_promo_12_day,item_store_promo_11_day,item_store_promo_10_day,item_store_promo_9_day,item_store_promo_8_day,item_store_promo_7_day,item_store_promo_6_day,item_store_promo_5_day,item_store_promo_4_day,item_store_promo_3_day,item_store_promo_2_day,item_store_promo_1_day,promo_day_in_15_days,last_promo_day_in_15_days,firt_promo_day_in_15_days,class_1,class_2,class_3,class_4,class_5,class_6,class_7,class_8,class_9,class_10,item_1,item_2,item_3,item_4,...,item_10,item_11,item_12,item_13,store_1,store_2,store_3,store_4,store_5,store_6,store_7,family_1,family_2,family_3,family_4,family_5,family_6,family_7,city_1,city_2,city_3,city_4,city_5,city_6,state_1,state_2,state_3,state_4,state_5,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,type_1,type_2,type_3,type_4,perishable
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1
4,0,0,0,25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0


In [38]:
#Merging all the data points
cv_x = cv_item_store_x.reset_index(drop = True).merge(cv_item_store_x1.reset_index(drop = True), left_index=True, right_index=True)

In [39]:
[cv_x[col].update((cv_x[col] - cv_x[col].min()) / (cv_x[col].max() - cv_x[col].min())) for col in cv_x.columns]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [40]:
cv_x.head()

,item_store_average_3_days,item_store_average_7_days,item_store_average_16_days,item_store_average_30_days,item_store_average_60_days,item_store_average_120_days,item_store_WMA_3_days,item_store_WMA_7_days,item_store_WMA_16_days,item_store_WMA_30_days,item_store_WMA_60_days,item_store_WMA_120_days,item_store_std_3_days,item_store_std_7_days,item_store_std_16_days,item_store_std_30_days,item_store_std_60_days,item_store_std_120_days,item_store_6avgdow_0_days,item_store_6avgdow_1_days,item_store_6avgdow_2_days,item_store_6avgdow_3_days,item_store_6avgdow_4_days,item_store_6avgdow_5_days,item_store_6avgdow_6_days,item_store_20avgdow_0_days,item_store_20avgdow_1_days,item_store_20avgdow_2_days,item_store_20avgdow_3_days,item_store_20avgdow_4_days,item_store_20avgdow_5_days,item_store_20avgdow_6_days,item_store_6WMAdow_0_days,item_store_6WMAdow_1_days,item_store_6WMAdow_2_days,item_store_6WMAdow_3_days,item_store_6WMAdow_4_days,item_store_6WMAdow_5_days,item_store_6WMAdow_6_days,item_store_20WMAdow_0_days,...,item_10,item_11,item_12,item_13,store_1,store_2,store_3,store_4,store_5,store_6,store_7,family_1,family_2,family_3,family_4,family_5,family_6,family_7,city_1,city_2,city_3,city_4,city_5,city_6,state_1,state_2,state_3,state_4,state_5,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,type_1,type_2,type_3,type_4,perishable
0,0.000000,0.054309,0.024038,0.016333,0.018921,0.020408,0.000000,0.042098,0.036277,0.023272,0.017970,0.019560,0.000000,0.181759,0.121195,0.098948,0.087125,0.100892,0.017225,0.018845,0.029519,0.027358,0.014394,0.000000,0.000000,0.015300,0.011361,0.041859,0.023863,0.010362,0.009413,0.005531,0.000320,0.015038,0.058239,0.054972,0.005971,0.000000,0.000000,0.017765,...,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0
1,0.000000,0.093470,0.102859,0.092354,0.101810,0.063570,0.000000,0.044502,0.095371,0.094734,0.095358,0.085600,0.000000,0.201655,0.172097,0.175088,0.180566,0.182779,0.140727,0.105246,0.082432,0.089238,0.014394,0.040525,0.164057,0.068035,0.071491,0.055104,0.053603,0.021605,0.039251,0.063008,0.132253,0.097425,0.139557,0.101392,0.017370,0.025416,0.093978,0.101160,...,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0
2,0.154317,0.130056,0.124467,0.121564,0.125445,0.131901,0.160150,0.146045,0.127996,0.124873,0.126313,0.129104,0.085253,0.145179,0.167332,0.173819,0.162651,0.199847,0.151706,0.184815,0.138628,0.130923,0.037209,0.103233,0.088571,0.148955,0.203973,0.137627,0.128354,0.028936,0.128692,0.128996,0.117218,0.190130,0.131802,0.147946,0.038232,0.114986,0.101484,0.145733,...,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0
3,0.179121,0.134933,0.139684,0.139067,0.161043,0.159294,0.194555,0.154889,0.135927,0.143402,0.151962,0.157177,0.116975,0.198377,0.206263,0.239915,0.210895,0.216842,0.163497,0.142935,0.167812,0.231275,0.065998,0.099109,0.144452,0.167630,0.147605,0.192916,0.221952,0.091506,0.110925,0.158945,0.166598,0.111013,0.141864,0.199408,0.050174,0.110407,0.178329,0.153814,...,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1
4,0.271843,0.289547,0.281806,0.268229,0.261008,0.278359,0.261407,0.272481,0.283117,0.278350,0.267680,0.267798,0.073657,0.101251,0.094564,0.159696,0.166431,0.201678,0.305567,0.273092,0.247401,0.227949,0.099420,0.292621,0.288148,0.310015,0.313388,0.297016,0.240397,0.127774,0.290320,0.313755,0.288494,0.320873,0.243690,0.247213,0.132675,0.281963,0.283993,0.304880,...,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0


In [41]:
print('Shape of train_x and corresponding train_y is {}'.format(cv_x.shape))

Shape of train_x and corresponding train_y is (167515, 149)


In [42]:
#Generating y_i for cv
cv_y = item_store_sales_df[[str(col)[0:10] for col in pd.date_range(cv_date, periods = 16)]].values

<h1> Preparing test data </h1>

In [43]:
#gathering sales featres
test_date = date(2017, 8, 16)
test_dict = feature_engg_sales(item_store_sales_df, test_date, 'item_store')
test_item_store_x = pd.DataFrame(test_dict, index = [i for i in range(len(list(test_dict.values())[0]))])
test_item_store_x.shape

(167515, 64)

In [44]:
test_item_store_x.head()

,item_store_average_3_days,item_store_average_7_days,item_store_average_16_days,item_store_average_30_days,item_store_average_60_days,item_store_average_120_days,item_store_WMA_3_days,item_store_WMA_7_days,item_store_WMA_16_days,item_store_WMA_30_days,item_store_WMA_60_days,item_store_WMA_120_days,item_store_std_3_days,item_store_std_7_days,item_store_std_16_days,item_store_std_30_days,item_store_std_60_days,item_store_std_120_days,item_store_6avgdow_0_days,item_store_6avgdow_1_days,item_store_6avgdow_2_days,item_store_6avgdow_3_days,item_store_6avgdow_4_days,item_store_6avgdow_5_days,item_store_6avgdow_6_days,item_store_20avgdow_0_days,item_store_20avgdow_1_days,item_store_20avgdow_2_days,item_store_20avgdow_3_days,item_store_20avgdow_4_days,item_store_20avgdow_5_days,item_store_20avgdow_6_days,item_store_6WMAdow_0_days,item_store_6WMAdow_1_days,item_store_6WMAdow_2_days,item_store_6WMAdow_3_days,item_store_6WMAdow_4_days,item_store_6WMAdow_5_days,item_store_6WMAdow_6_days,item_store_20WMAdow_0_days,item_store_20WMAdow_1_days,item_store_20WMAdow_2_days,item_store_20WMAdow_3_days,item_store_20WMAdow_4_days,item_store_20WMAdow_5_days,item_store_20WMAdow_6_days,item_store_has_sale_day_3,item_store_has_sale_day_7,item_store_has_sale_day_16,item_store_has_sale_day_30,item_store_has_sale_day_60,item_store_has_sale_day_120,item_store_last_has_sale_day_3,item_store_last_has_sale_day_7,item_store_last_has_sale_day_16,item_store_last_has_sale_day_30,item_store_last_has_sale_day_60,item_store_last_has_sale_day_120,item_store_first_has_sale_day_3,item_store_first_has_sale_day_7,item_store_first_has_sale_day_16,item_store_first_has_sale_day_30,item_store_first_has_sale_day_60,item_store_first_has_sale_day_120
0,0.000000,0.099021,0.361296,0.275522,0.160866,0.164679,0.000000,0.102557,1.621129,3.929400,6.328366,10.091264,0.000000,0.261985,0.496665,0.478507,0.375651,0.355483,0.115525,0.298627,0.231049,0.183102,0.183102,0.366204,0.000000,0.138629,0.158903,0.283148,0.158903,0.124245,0.179176,0.034657,0.467599,1.324251,0.473100,0.374923,0.741127,1.299153,0.000000,1.456269,2.375802,2.023253,1.364850,1.682490,2.166026,0.173452,0,1,6,8,10,23,3,6,6,6,6,6,0,6,16,26,52,117
1,0.000000,0.156945,0.180648,0.331321,0.444620,0.439287,0.000000,0.005605,1.316549,2.848349,10.284561,27.572157,0.000000,0.415236,0.397677,0.520680,0.605854,0.580522,0.549306,0.346574,0.760725,0.414151,0.115525,0.298627,0.346574,0.517255,0.436162,0.407393,0.391601,0.144519,0.343968,0.394828,1.490974,0.247553,1.211101,0.895971,0.005501,0.862153,0.247553,6.172255,4.557253,4.982854,4.779882,1.419712,3.765010,4.486219,0,1,3,10,25,50,3,7,7,7,7,7,0,7,11,30,60,98
2,0.231049,0.495105,0.631845,0.714515,0.756274,0.829553,0.038508,1.106914,3.849658,9.403114,21.304909,46.340178,0.400189,0.523970,0.572061,0.541249,0.566843,0.615813,0.577623,1.268145,1.085043,0.808672,0.231049,0.529676,0.828302,0.922618,1.204500,0.974720,0.843555,0.228217,0.695809,0.877648,1.182751,2.959446,2.427112,1.886901,0.819674,1.016996,2.022990,7.934720,11.044125,9.762159,8.285875,2.150521,5.958659,7.603378,1,4,10,21,43,88,3,3,3,3,3,3,3,6,16,30,60,120
3,0.462098,0.980990,1.071718,1.017638,1.001216,1.037053,0.770164,1.767097,6.795309,14.848898,29.619859,61.225990,0.400189,0.918099,0.733244,0.702227,0.733502,0.697977,1.480306,0.865493,1.281714,1.066155,0.414151,0.981017,1.096542,1.111133,0.935184,1.268461,1.468864,0.586569,0.871758,1.041818,3.985686,2.807968,3.356871,2.145889,1.262175,2.442531,2.195283,11.416194,8.603803,12.708312,13.280417,5.106024,8.489436,9.013624,2,5,13,23,45,93,1,1,1,1,1,1,2,7,16,29,60,120
4,0.998577,1.560437,1.663453,1.714960,1.686812,1.719123,1.701486,4.047843,11.601219,23.591361,49.668833,100.670846,0.871961,0.755495,0.697497,0.588849,0.622479,0.618062,2.035673,2.014368,1.894527,1.767434,0.682391,1.787064,1.894457,2.128771,1.853582,1.926297,1.647805,0.820055,2.065504,1.916016,5.217690,5.143940,4.723765,4.517536,1.098650,4.270604,4.860144,19.699817,17.574288,18.098191,15.471052,6.462646,18.369778,17.5

In [45]:
test_dict = feature_engg_promo(item_store_promo_df, class_array, store_array, test_date, 'item_store')
test_item_store_x1 = pd.DataFrame(test_dict, index = [i for i in range(len(list(test_dict.values())[0]))])
test_item_store_x1.shape

(167515, 85)

In [46]:
test_item_store_x1.shape

(167515, 85)

In [47]:
test_x = test_item_store_x.reset_index(drop = True).merge(test_item_store_x1.reset_index(drop = True), left_index=True, right_index=True)

In [48]:
[test_x[col].update((test_x[col] - test_x[col].min()) / (test_x[col].max() - test_x[col].min())) for col in test_x.columns]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [49]:
test_x.shape

(167515, 149)

In [50]:
print('Shape of train_x and corresponding train_y is {}'.format(test_x.shape))

Shape of train_x and corresponding train_y is (167515, 149)


In [51]:
print(train_x.shape, train_y.shape)
print(cv_x.shape, cv_y.shape)
print(test_x.shape)

(1340120, 149) (1340120, 16)
(167515, 149) (167515, 16)
(167515, 149)


#Modelling

<h1> Linear Regression </h1>

We have 16 steps to predict and we have collected our y such that it is a vector of Mx16, so we will train x for each of these y and based on the result for every y we will generate the forecast

In [ ]:
test_pred = []
for i in range(train_y.shape[1]):
    print('step{}'.format(i+1))
    lr = LinearRegression()
    lr.fit(train_x, train_y[: , i])
    test_pred.append(lr.predict(test_x))

step1
step2
step3
step4
step5
step6
step7
step8
step9
step10
step11
step12
step13
step14
step15
step16


In [ ]:
#Creating prediction df
y_test = np.array(test_pred).transpose()
pred_df = pd.DataFrame(y_test, columns=pd.date_range("2017-08-16", periods=16))

In [ ]:
pred_df.head()

,2017-08-16,2017-08-17,2017-08-18,2017-08-19,2017-08-20,2017-08-21,2017-08-22,2017-08-23,2017-08-24,2017-08-25,2017-08-26,2017-08-27,2017-08-28,2017-08-29,2017-08-30,2017-08-31
0,0.210249,0.223341,0.289090,0.270891,0.210922,0.250148,0.215341,0.262398,0.232133,0.280765,0.253660,0.199466,0.234750,0.202726,0.250296,0.229712
1,0.317271,0.300324,0.379285,0.318358,0.118592,0.314419,0.322006,0.400568,0.322340,0.356570,0.239060,0.080438,0.249397,0.232299,0.332197,0.304362
2,0.714128,0.769529,0.864926,0.703121,0.287471,0.621062,0.693032,0.759970,0.796433,0.873410,0.685914,0.252756,0.602160,0.667950,0.738206,0.780200
3,1.016627,0.975421,1.213480,1.213211,0.711763,0.852536,0.844865,0.994915,0.971457,1.199146,1.153113,0.593824,0.813760,0.793996,0.928450,0.927661
4,1.903333,1.759549,1.925219,1.665465,1.139938,1.764955,1.755045,1.879388,1.731188,1.840615,1.502486,0.983890,1.634322,1.623717,1.753439,1.652570


In [ ]:
item_store_sales_df['store_nbr'] = pd.to_numeric(item_store_sales_df['store_nbr'])
items_df['class'] = pd.to_numeric(items_df['class'])

In [ ]:
#Melting down the predicted values based on dates
pred_df = item_store_sales_df[['item_nbr', 'store_nbr']].merge(pred_df, left_index=True, right_index=True)
pred_df = pred_df.melt(id_vars=['item_nbr', 'store_nbr'], var_name='date', value_name='unit_sales')
pred_df = pred_df.merge(items_df[['item_nbr', 'class']], how = 'left', on = 'item_nbr')
pred_df['unit_sales'] = pred_df['unit_sales'].apply(lambda x : np.expm1(x))

In [ ]:
pred_df.head()

,item_nbr,store_nbr,date,unit_sales,class
0,96995,1,2017-08-16,0.233986,1093
1,99197,1,2017-08-16,0.373375,1067
2,103520,1,2017-08-16,1.042405,1028
3,103665,1,2017-08-16,1.763855,2712
4,105574,1,2017-08-16,5.708217,1045


In [ ]:
#Reading test_file
test_df = pd.read_csv('test.csv')
test_df['date'] = pd.to_datetime(test_df['date'])

In [ ]:
#Merging with the predicted values
test_df = test_df.merge(pred_df[['item_nbr', 'store_nbr', 'date', 'unit_sales']], on = ['date', 'store_nbr', 'item_nbr'], how = 'left')
test_df['unit_sales'] = test_df['unit_sales'].clip(lower = 0)
#Filling null values with 0
test_df = test_df.fillna(0)
#Making submission file
test_df[['id', 'unit_sales']].to_csv('lr_submission.csv', index = False)

In [ ]:
test_df[['id', 'unit_sales']].head()

,id,unit_sales
0,125497040,0.233986
1,125497041,0.373375
2,125497042,0.000000
3,125497043,1.042405
4,125497044,1.763855


In [ ]:
del test_df, pred_df

**Since we know that perishable items have more weights in our scoring method as compared to non perishable, hence we are creating a weight vector with perishable items having a weight of 1.25 and others having a weight of 1, this will be used by XGBoost to give more efforts with items with higher weights.**

In [52]:
train_weights = pd.concat([pd.DataFrame(item_store_sales_df['item_nbr']).merge(items_df[['item_nbr', 'perishable']], on = 'item_nbr', how = 'left')['perishable']] * 8)*0.25 + 1
cv_weights = pd.DataFrame(item_store_sales_df['item_nbr']).merge(items_df[['item_nbr', 'perishable']], on = 'item_nbr', how = 'left')['perishable'] * 0.25 + 1

In [ ]:
train_weights.shape, cv_weights.shape

((1340120,), (167515,))

In [53]:
train_weights.head()

0    1.00
1    1.00
2    1.00
3    1.25
4    1.00
Name: perishable, dtype: float64

In [54]:
cv_weights.head()

0    1.00
1    1.00
2    1.00
3    1.25
4    1.00
Name: perishable, dtype: float64

<h1> XGBoost without tuned parameters </h1>

In [ ]:
test_pred = []
for i in range(train_y.shape[1]):
    print('step{}'.format(i+1))
    start_time = time.time()
    xg = XGBRegressor()
    xg.fit(train_x, train_y[: , i], sample_weight = train_weights.values)
    test_pred.append(xg.predict(test_x))
    print('done in {}'.format(time.time() - start_time))

step1
done in 331.6978657245636
step2
done in 343.8066828250885
step3
done in 342.00068831443787
step4
done in 340.25216579437256
step5
done in 340.5731554031372
step6
done in 339.94398260116577
step7
done in 340.4451413154602
step8
done in 341.4083557128906
step9
done in 340.8268074989319
step10
done in 341.2782769203186
step11
done in 340.4570393562317
step12
done in 340.13201689720154
step13
done in 340.543958902359
step14
done in 339.9647686481476
step15
done in 339.2215938568115
step16
done in 341.0175998210907


In [55]:
#Generating prediction df
y_test = np.array(test_pred).transpose()
pred_df = pd.DataFrame(y_test, columns=pd.date_range("2017-08-16", periods=16))
pred_df.head(10)

NameError: ignored

In [ ]:
item_store_sales_df['store_nbr'] = pd.to_numeric(item_store_sales_df['store_nbr'])
items_df['class'] = pd.to_numeric(items_df['class'])

In [ ]:
#Melting based on dates and adding other columns
pred_df = item_store_sales_df[['item_nbr', 'store_nbr']].merge(pred_df, left_index=True, right_index=True)
pred_df = pred_df.melt(id_vars=['item_nbr', 'store_nbr'], var_name='date', value_name='unit_sales')
pred_df = pred_df.merge(items_df[['item_nbr', 'class']], how = 'left', on = 'item_nbr')
pred_df['unit_sales'] = pred_df['unit_sales'].apply(lambda x : np.expm1(x))
pred_df.head()

,item_nbr,store_nbr,date,unit_sales,class
0,96995,1,2017-08-16,0.268535,1093
1,99197,1,2017-08-16,0.302717,1067
2,103520,1,2017-08-16,1.264865,1028
3,103665,1,2017-08-16,2.325738,2712
4,105574,1,2017-08-16,6.268309,1045


In [ ]:
#Loading test file
test_df = pd.read_csv('test.csv')
test_df['date'] = pd.to_datetime(test_df['date'])

In [ ]:
#Merging with predicted results and saving submission file
test_df = test_df.merge(pred_df[['item_nbr', 'store_nbr', 'date', 'unit_sales']], on = ['date', 'store_nbr', 'item_nbr'], how = 'left')
test_df['unit_sales'] = test_df['unit_sales'].clip(lower = 0)
test_df = test_df.fillna(0)
test_df[['id', 'unit_sales']].to_csv('xg_submission.csv', index = False)

In [ ]:
test_df[['id', 'unit_sales']].head(10)

,id,unit_sales
0,125497040,0.268535
1,125497041,0.302717
2,125497042,0.000000
3,125497043,1.264865
4,125497044,2.325738
5,125497045,6.268309
6,125497046,13.093959
7,125497047,0.000000
8,125497048,0.668626
9,125497049,0.324347


In [ ]:
del test_df, pred_df

<h1> hyperparameter tuning using RandomizedSearchCv </h1>

In [ ]:
def random_search(x, y, x_cv, y_cv):
    '''
    This function is called during each step and it returns best parameter and best estimator during each step
    '''
    params = {'max_depth' : [2, 4, 6, 8, 10],
          'learning_rate' : [0.1, 0.2, 0.3],
          'n_estimators' : [5, 10, 50, 100]
         }
    clf = XGBRegressor(objective = 'reg:squarederror', eval_metric = 'rmse')
    rv = RandomizedSearchCV(clf, param_distributions = params, n_iter = 8, scoring = 'neg_root_mean_squared_error', n_jobs = -1, verbose = 3, cv = 3)
    rv.fit(x, y)
    return rv.best_estimator_, rv.best_params_

In [ ]:
#Tuning parameter and saving the best estimator and parameters
test_pred = []
for i in range(train_y.shape[1]):
    print('step{}'.format(i+1))
    xg, best_params = random_search(train_x, train_y[:, i], cv_x, cv_y[:, i])
    dump(xg, 'clf_step_{}.joblib'.format(i+1))
    dump(best_params, 'para_step_{}.joblib'.format(i+1))

step1
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 11.7min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed: 31.0min finished


step2
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 45.8min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed: 95.6min finished


step3
Fitting 5 folds for each of 8 candidates, totalling 40 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 15.6min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed: 66.6min finished


step15
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 32.9min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed: 75.6min finished


step16
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed: 53.9min finished


<h1> Using XGBoost with tuned parameter to generate final results </h1>

In [ ]:
def load_param(step):
    '''
    This function loads the best parameters used by XGBoost
    '''
    best_param = load('/home/jupyter/final_para/para_step_{}.joblib'.format(step))
    print(best_param)
    params = {}
    params['objective'] = 'reg:squarederror'
    params['eval_metric'] = 'rmse'
    params['eta'] = 0.02
    #params['n_estimators'] = best_param['n_estimators']
    params['max_depth'] = best_param['max_depth']
    params['learning_rate'] = best_param['learning_rate']
    
    return params

In [ ]:
import xgboost as xgb
test_pred = []
dtest = xgb.DMatrix(test_x)
for i in range(train_y.shape[1]):
    param = load_param(i + 1)
    #print(param)
    print('step{}'.format(i+1))
    dtrain = xgb.DMatrix(train_x, label = train_y[:, i], weight = train_weights)
    dval = xgb.DMatrix(cv_x, label = cv_y[:, i], weight = cv_weights)
    
    watchlist = [(dtrain, 'train'), (dval, 'val')]
    model = xgb.train(param, dtrain, 500, watchlist, early_stopping_rounds = 20, verbose_eval = 10)
    
    test_pred.append(model.predict(dtest))

{'n_estimators': 50, 'max_depth': 8, 'learning_rate': 0.1}
step1
[0]	train-rmse:1.09407	val-rmse:1.05591
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 20 rounds.
[10]	train-rmse:0.64464	val-rmse:0.61657
[20]	train-rmse:0.55929	val-rmse:0.54722
[30]	train-rmse:0.54389	val-rmse:0.54058
[40]	train-rmse:0.53921	val-rmse:0.54024
[50]	train-rmse:0.53623	val-rmse:0.53984
[60]	train-rmse:0.53388	val-rmse:0.53917
[70]	train-rmse:0.53215	val-rmse:0.53901
[80]	train-rmse:0.53080	val-rmse:0.53890
[90]	train-rmse:0.52960	val-rmse:0.53882
[100]	train-rmse:0.52858	val-rmse:0.53873
[110]	train-rmse:0.52758	val-rmse:0.53867
[120]	train-rmse:0.52655	val-rmse:0.53866
[130]	train-rmse:0.52558	val-rmse:0.53863
[140]	train-rmse:0.52468	val-rmse:0.53876
Stopping. Best iteration:
[128]	train-rmse:0.52577	val-rmse:0.53859

{'n_estimators': 100, 'max_depth': 8, 'learning_rate': 0.1}
step2
[0]	train-rmse:1.02810	val-rmse:1.00130


In [ ]:
#Creating prediction df
y_test = np.array(test_pred).transpose()
pred_df = pd.DataFrame(y_test, columns=pd.date_range("2017-08-16", periods=16))
pred_df.head()

,2017-08-16,2017-08-17,2017-08-18,2017-08-19,2017-08-20,2017-08-21,2017-08-22,2017-08-23,2017-08-24,2017-08-25,2017-08-26,2017-08-27,2017-08-28,2017-08-29,2017-08-30,2017-08-31
0,0.211937,0.176730,0.259274,0.234703,0.137470,0.206698,0.206255,0.219694,0.199748,0.245661,0.257121,0.170452,0.215059,0.194608,0.200388,0.185197
1,0.318681,0.320026,0.323715,0.375148,0.145478,0.303089,0.305589,0.333401,0.275312,0.356439,0.343567,0.191223,0.333231,0.324571,0.362044,0.338342
2,0.790881,0.766193,0.904757,0.819249,0.243183,0.683963,0.748915,0.821975,0.812995,0.937696,0.828848,0.276371,0.697604,0.765209,0.819312,0.773956
3,1.180265,1.016764,1.299401,1.288223,0.580249,0.952050,1.048379,1.133404,0.982418,1.315754,1.297190,0.555794,0.973711,1.032635,1.086919,0.962885
4,2.059678,1.869376,1.993411,1.624396,0.901049,1.838199,1.895689,2.003506,1.824216,2.017774,1.684260,0.871924,1.820947,1.857037,2.001970,1.735209


In [ ]:
item_store_sales_df['store_nbr'] = pd.to_numeric(item_store_sales_df['store_nbr'])
items_df['class'] = pd.to_numeric(items_df['class'])

In [ ]:
#melting the predicted result based on dates
pred_df = item_store_sales_df[['item_nbr', 'store_nbr']].merge(pred_df, left_index=True, right_index=True)
pred_df = pred_df.melt(id_vars=['item_nbr', 'store_nbr'], var_name='date', value_name='unit_sales')
pred_df = pred_df.merge(items_df[['item_nbr', 'class']], how = 'left', on = 'item_nbr')
pred_df['unit_sales'] = pred_df['unit_sales'].apply(lambda x : np.expm1(x))
pred_df.head()

,item_nbr,store_nbr,date,unit_sales,class
0,96995,1,2017-08-16,0.236070,1093
1,99197,1,2017-08-16,0.375312,1067
2,103520,1,2017-08-16,1.205339,1028
3,103665,1,2017-08-16,2.255238,2712
4,105574,1,2017-08-16,6.843444,1045


In [ ]:
test_df = pd.read_csv('test.csv')
test_df['date'] = pd.to_datetime(test_df['date'])

In [ ]:
#Results merged with test file and submission file created
test_df = test_df.merge(pred_df[['item_nbr', 'store_nbr', 'date', 'unit_sales']], on = ['date', 'store_nbr', 'item_nbr'], how = 'left')
test_df['unit_sales'] = test_df['unit_sales'].clip(lower = 0)
test_df = test_df.fillna(0)
test_df[['id', 'unit_sales']].to_csv('xg_submission2.csv', index = False)

In [ ]:
test_df[['id', 'unit_sales']].head()

,id,unit_sales
0,125497040,0.236070
1,125497041,0.375312
2,125497042,0.000000
3,125497043,1.205339
4,125497044,2.255238


#Conclusions

In [56]:
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["Model", "Private Score", "Rank"]

x.add_row(["Baseline - 16days MA", .59249, 1197])
x.add_row(["Linear Regression", .53398, 728])
x.add_row(["XGBoost Regressor", .52293, 334])
x.add_row(["XGBoost with tuned parameters", .52026, 103])

print(x)

+-------------------------------+---------------+------+
|             Model             | Private Score | Rank |
+-------------------------------+---------------+------+
|      Baseline - 16days MA     |    0.59249    | 1197 |
|       Linear Regression       |    0.53398    | 728  |
|       XGBoost Regressor       |    0.52293    | 334  |
| XGBoost with tuned parameters |    0.52026    | 103  |
+-------------------------------+---------------+------+


#Future Scope:

1. Information from transaction file, holiday file and oil file is still not explored in the model.
2. Sales/promo features are used at item-store level, but information at item/ store/ item-class level, may give us better results, although tried using them but the model results were not great, may be if these features used differently can give better result.
3. We can use LSTM to make prediction and see if our results improve further, but for that we need to come up with the correct architecture. For now, XGBoost is doing a good job, so sticking with it.